In [1]:
import sys
import numpy as np
import time
import pickle
import pandas as pd
import copy
import matplotlib.pyplot as plt

import allensdk
import pandas as pd

In [2]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

# Mouse Connectivity

This notebook analyzes the mouse connectivity data for the purpose of calculating the information flow in the brain. We reply on Allen brain Atlas API  (allensdk package) with some custom codes.

As a brief summary, Allen brain Atlas: Mouse Brain Connectivity contains data on  experiments on the mouse brain. Each experiment has an associated (unique) injection site where green fluroescent protein virus is injected. This injection infects the cell-bodies of the neurons in the injection site, and they propagates along the axons (carrier of the action potential of neurons) where they reach its terminal and emit green lights. These signals then are imaged through green-flurescent imaging and mapped accordingly. More information can be found on0 http://connectivity.brain-map.org/. Some defintions relevant to the throughout the notbook are defined as follows.

projection density = sum of detected pixels / sum of all pixels in division  
projection energy = sum of detected pixel intensity / sum of all pixels in division  
injection_fraction = fraction of pixels belonging to manually annotated injection site  
injection_density = density of detected pixels within the manually annotated injection site  
injection_energy = energy of detected pixels within the manually annotated injection site  
data_mask = binary mask indicating if a voxel contains valid data (0=invalid, 1=valid). Only valid voxels should be used for analysis  


This notebook carefully combines multiple experiments to create a connectome data. This data is then traslated into the information flow data, namely we determine how much information flows at a regional level.

## Fetch all the Experiment list


First we create MouseConnectivityCache instance.

Fome this we may fetch the experiment list. One in dataFrame and one in dictionary format.

In [3]:
# MouseConnectivityCache allows access to data stored within
mcc=MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')

#get the id of all experiments.
#Create one for Pandas dataframe and the other in terms of dictionary
experiment_list = mcc.get_experiments(dataframe=True)
experiment_list_dict=mcc.get_experiments(dataframe=False)

Sample of some of the entries of the experiment_list looks like:

In [4]:
experiment_list[:3]

,gender,id,injection_structures,injection_volume,injection_x,injection_y,injection_z,primary_injection_structure,product_id,specimen_name,strain,structure_abbrev,structure_id,structure_name,transgenic_line,transgenic_line_id
id,,,,,,,,,,,,,,,,
527712447,F,527712447,"[502, 926, 1084, 484682470]",0.006655,9240,3070,8990,502,5,Penk-IRES2-Cre-neo-249961,C57BL/6J,SUB,502,Subiculum,Penk-IRES2-Cre-neo,298725927.0
301875966,M,301875966,"[574, 931]",0.105746,9170,6850,6200,574,5,Gabrr3-Cre_KC112-3467,C57BL/6J,PG,931,Pontine gray,Gabrr3-Cre_KC112,177838877.0
520336173,M,520336173,"[1, 210, 491, 525, 1004]",0.025762,7810,6550,6450,1,5,Hdc-Cre_IM1-204103,None,TMv,1,"Tuberomammillary nucleus, ventral part",Hdc-Cre_IM1,177839494.0


where the columns are:

Each experiment entry contains the following columns:

In [5]:
for i in experiment_list_dict[0].keys():
    print(i)

injection_y
injection_x
injection_z
product_id
structure_name
gender
transgenic_line
injection_volume
structure_abbrev
transgenic_line_id
primary_injection_structure
strain
id
specimen_name
injection_structures
structure_id


with the number of separate experiments being:

In [6]:
len(experiment_list_dict)

2995

This is for allensdk 0.14.5. Number of experiments may vary depending on the version of the package. Note that each injection sites has a unique experiemnt, that is, no more than one injection sites were allowed in a single mouse brain.

## Annotation Volume

We then download annotation volume for future uses. This comes in the form of an array where structures are denoted by the values of the elements. Spatial coordinates on the other hand, are represented by the indices of the array. First create an instance using the MouseConnectivityCache (mcc)


In [7]:
annt=mcc.get_annotation_volume()[0]

the annotation volume has the shape

In [8]:
annt.shape

(528L, 320L, 456L)

for future, we collect the id of all the structures present in the annt volume

In [10]:
annt_structures=[]
for i in range(annt.shape[0]):
    for j in range(annt.shape[1]):
        for k in range(annt.shape[2]):
            if annt[i,j,k] not in annt_structures:
                annt_structures.append(annt[i,j,k])


## Structure_tree

Allenmouse structure ontology is organized as a tree where each edge indicates the physical containment. Structuretree can be used to fetch properties of a given structure, including its id, name, acronym, etc..

In [9]:
structure_tree=mcc.get_structure_tree()

sample of each structure is shown:

In [10]:
pd.DataFrame(structure_tree.get_structures_by_name(['Thalamus','Hypothalamus']))

,acronym,graph_id,graph_order,id,name,rgb_triplet,structure_id_path,structure_set_ids
0,TH,1,641,549,Thalamus,"[255, 112, 128]","[997, 8, 343, 1129, 549]","[2, 112905828, 691663206, 12, 184527634, 11290..."
1,HY,1,715,1097,Hypothalamus,"[230, 68, 56]","[997, 8, 343, 1129, 1097]","[2, 112905828, 691663206, 12, 184527634, 11290..."


where the columns are:

In [11]:
for i in structure_tree.get_structures_by_name(['Thalamus'])[0]:
    print(i)

rgb_triplet
graph_id
name
acronym
graph_order
structure_id_path
structure_set_ids
id


## structure set

On top of strucure tree, AllenmouseSDK provides a convenient way to group structures together through the structure sets.
For example, if i wish to fetch structures trees beonging to Midbrain, I can easily do so.

In [12]:
from allensdk.api.queries.ontologies_api import OntologiesApi

#initialize oapi_inst instance
oapi_inst=OntologiesApi()

In [13]:
structure_set_ids=structure_tree.get_structure_sets()
structure_set=pd.DataFrame(oapi_inst.get_structure_sets(structure_set_ids))
structure_set

,description,id,name
0,List of structures in Isocortex layer 5,667481446,Isocortex layer 5
1,List of structures in Isocortex layer 6b,667481450,Isocortex layer 6b
2,Summary structures of the cerebellum,688152368,Cerebellum
3,List of structures for ABA Differential Search,12,ABA - Differential Search
4,List of valid structures for projection target...,184527634,Mouse Connectivity - Target Search
5,Structures whose surfaces are represented by a...,691663206,Mouse Brain - Has Surface Mesh
6,Summary structures of the midbrain,688152365,Midbrain
7,Summary structures of the medulla,688152367,Medulla
8,Summary structures of the striatum,688152361,Striatum
9,Structures representing subdivisions of the mo...,687527945,Mouse Connectivity - Summary


As shown above, we can select the structure set based on its description and fetch the desired collection of structures>

Here our interest is in the projection research, therefore we fetch the summary structures from 184527634 (Mouse Connectivity - Target Search)

In [14]:
projection_structures_target=pd.DataFrame(structure_tree.get_structures_by_set_id([184527634]))

### how many structures in each?

Out of curiosity, we explore how many structures are contained in each structure set:

In [15]:
#very clever name for a variable indicating exactly what it is used for.
printme={'experiment id':[],'number of structures':[]}

for i in structure_set[structure_set.columns[1]]:  #column 1 is id
    printme['experiment id'].append(i)
    printme['number of structures'].append(len(structure_tree.get_structures_by_set_id([i])))
print pd.DataFrame(printme)

    experiment id  number of structures
0       667481446                    43
1       667481450                    43
2       688152368                    18
3              12                   843
4       184527634                   840
5       691663206                   840
6       688152365                    39
7       688152367                    45
8       688152361                    14
9       687527945                   293
10      688152359                    15
11      514166994                     7
12      688152358                    11
13      167587189                   316
14      667481445                    26
15      687527670                    12
16      688152362                     9
17      114512892                    79
18      112905813                   111
19             10                    74
20      112905828                   397
21      667481449                    43
22              3                    51
23      667481440                    43


# Maps

Each structure can be denoted by all kinds of variables such as its id, acronym, or its full name. Therefore it is convenient to define maps that will take care of this. We want the map to function as a converter between variables.


In [16]:
projection_structures_inj=pd.DataFrame(projection_structures_target)

acronym=projection_structures_inj['acronym'].values.tolist()
acronym_to_id={}
acronym_to_name={}
acronym_to_volume={}
id_to_idpath={}
id_to_name={}

for i in range(len(acronym)):
    a=projection_structures_inj['acronym'][i] #structure acronym
    b=projection_structures_inj['structure_id_path'][i][-1] #sturcture id
    c=projection_structures_inj['name'][i] #structure name
    d=projection_structures_inj['structure_id_path'][i] #structure id path
    acronym_to_id[a]=b
    acronym_to_name[a]=c
    id_to_name[b]=c
    id_to_idpath[b]=d


### structure volumes

as far as I am aware, getting the structure volume is not straight forward

define inverse map for the dictionary

In [17]:
def invert_me(arg):
    """inverts a map
        args:
            arg(dictionary): input dictionary
        returns:
            inverted input dictionary"""
    return {v: k for k,v in arg.iteritems()}

In [18]:
id_to_acronym=invert_me(acronym_to_id)
name_to_acronym=invert_me(acronym_to_name)
name_to_id=invert_me(id_to_name)

Since the structures organized as trees, it would be nice to have a dictionary that can return all its daughter structures (or all structures beneath it). Hence here we define a dictionary that contains its direct substructures (only one level below): Substr

In [19]:
projection_structures_inj.columns[-2]

'structure_id_path'

In [20]:
Substr={}
Substr[997]=[]
for acr in acronym:
    Substr[acronym_to_id[acr]]=[]

    
unknown_structures=[]


for i in projection_structures_inj['structure_id_path']:
    try:
        Substr[i[-2]].append(i[-1])
    except:
        unknown_structures.append(i[-2])

In [21]:
unknown_structures

[81, 81]

Unknown structure with id 81

Now we define functions that unpacks a list of all structures below a particular structure using Substr

In [22]:
def Unpack(structure_id,MST,include_self=False):
    """returns a list of substructures of a given structure
        args:
            structures_id(int): id of the structure
            MST(dictionary): My Structure Tree
            include_self(bool): decide whether to include the initial structure, default=False
        
        returns:
            king(list) """
    #Check for Error
    #if int(structure_id)!=structure_id:
    #    raise ValueError("not a valid structure_id. structure_id must be an integer")
    
    
    unpacked=[]
    
    
    
    def Unpack_(structure_id,MST):
        """Primarily used for 'recursion' of the unpack"""
        
        #check error
        #if structure_id  in structure_id_list_exception:
        #    raise ValueError("structure id not recognized")
         
        leng=len(MST[structure_id])

        if leng>=1:
            unpacked.append(structure_id)
            for i in MST[structure_id]:
                Unpack_(i,MST)

        elif leng==0:
            unpacked.append(structure_id)
        

    Unpack_(structure_id,MST)
    
    if include_self:
        pass
    elif not include_self:
        unpacked.remove(structure_id)
    
    return unpacked

In [23]:
name_to_id['Thalamus']

549

In [24]:
len(Unpack(997,Substr,include_self=False))

838

## Injection Fraction 

We wish to find how many fractions of the total injection volume of an experiment belong to the structure of interest.

id_to_idpath defined earlier is not complete without structure id 997 and 0

In [386]:
#id_to_idpath[0]=[]
#id_to_idpath[997]=[997]

structure_id_list_exception=[81]

In [387]:
def create_structure_dictionary(annt_values,inj_density_values,include_density=True):
    """args:
        annt_values: (array, int 32): array of structure ids obtained from annotation volume[injection_mask]
        inj_density_values (array): 
        
        returns:
            structuer_dict(dictionary): a map from a structure id to its fraction in the injection volume
            """
    
    #check that annt_values and inj_density are the same size
    if len(annt_values)!=len(inj_density_values):
        raise ValueError('annt volume and inj density must be the same size')
    
    structure_dict={}
    leng=len(annt_values)
    exception_count=0
    exception_id_list=[]
    
    if include_density:
        
        for i in range(leng):
            
            if annt_values[i] in structure_id_list_exception:
                exception_id_list.append(annt)
                pass
            elif annt_values[i] not in structure_id_list_exception:
                if annt_values[i] not in structure_dict:
                    structure_dict[annt_values[i]]=0
                elif annt_values[i] in structure_dict:
                    structure_dict[annt_values[i]]+=inj_density_values[i]
                    
                    
                    
                    
                    
        #need to add the id of each superstructures of structure_dict
        extra=[]
        
        for i in structure_dict:
            try:
                sup=id_to_id_path[i]
                for j in sup:
                    if j not in structure_dict:
                        extra.append(j)
            except:
                exception_count+=1
                pass
        for i in extra:
            structure_dict[i]=0
                
        return structure_dict,exception_count,exception_id_list
    
    
    elif not include_density:
        for i in arg:
            try:
                if i not in structure_dict:
                    structure_dict[i]=0
            except:
                exception_count+=1
                pass

            if i in structure_dict:
                structure_dict[i]+=1
        return structure_dict,exception_count,exception_id_list
    
def generate_experiment_structure_fractions( experiment_list, annt,MST,include_density=True):
    """returns a dictionary that contains fraction information about the injection volume of the structure"""
    
    """args:
        experiment_list(list): list of experiment ids
        annt (ndarray) annotation volume
        include_density (bool): decide whether to calculate using the density of simple volume number.
        Default=True"""
    
    #returning dictionary
    return_dict={}
    
    #iterate over the given experiment list.
    for i in experiment_list:
        
        #fetch injection density information
        inj_density=mcc.get_injection_density(i)[0]
        #create booleean array that is True on the injection sites
        inj_density_bool=inj_density!=0.0
        
        #np.where returns a tuple (x,y,z) for 3d, which are to be unpacked immediately
        a,b,c=np.where(inj_density_bool)
        #using the voxel coordinates a,b,c, find density values at each coordinate. 
        inj_density_values=inj_density[a,b,c]
        #and annotation values that contain id list
        annt_values=annt[a,b,c]
        
        
        
        #call the function create_structure_dictionary that analyzies the fraction information
        sub_dict,exception_count_sub,exception_id_list_sub=create_structure_dictionary(annt_values,inj_density_values,include_density=True)
        
        
        
        for p in sub_dict:
            if not include_density:
                leng=annt_values.shape[0]
                sub_dict[p]=float(sub_dict[p])/leng
            elif include_density:
                #normalize the fraction
                sub_dict[p]=float(sub_dict[p])/sum(inj_density_values)
        
        return_dict[i]=sub_dict
    
    exception_count=0
    exception_id_list=[]
    
    # now add the fraction of substructures to the superstructure
    #normal copy would not really 'copy' the dictionary. Thus use copy.deepcopy
    return_dict_modified=copy.deepcopy(return_dict)
    for i in return_dict:
        for j in return_dict[i]:
            #call unpack function to list all the substructures
            if j in projection_structures_inj:
                unpackedpre=Unpack(j,MST,include_self=True)

                for k in unpackedpre:
                    if j!=k:
                        try:
                            aa=return_dict[i][k].copy()
                            return_dict_modified[i][j]+=aa
                        except:
                            exception_count+=1
                            pass
            else:
                exception_count+=1
                exception_id_list.append(j)
                pass
    return return_dict_modified,exception_count,exception_id_list,exception_count_sub,exception_id_list_sub

def map_to_acronym(dict_):
    return_dict={}
    exception_count,exception_id=(0,[])
    for i in dict_:
        try:
            return_dict[id_to_acronym[i]]=dict_[i]
        except:
            exception_count+1
            exception_id.append(i)
            pass
    return return_dict

In [39]:
run=raw_input("Computationally intensive (roughly 6-8 hours) Run? (y/n)")
if run=="y":
    Master_injection_fraction_dictionary={}
    ec=0
    ecl=[]

    for i in experiment_list['id'].tolist():
        ecp,ec,ecl,ecs,ecls=generate_experiment_structure_fractions([i],annt,Substr,include_density=True)
        Master_injection_fraction_dictionary[i]=ecp[i]
        
elif run=="n":
    print "operation cancelled"


Computationally intensive (roughly 6-8 hours) Run? (y/n)y


2018-08-01 20:21:08,697 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/310035922?image=injection_density&resolution=25
2018-08-01 20:23:52,525 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/313397385?image=injection_density&resolution=25
2018-08-01 20:29:23,744 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306561747?image=injection_density&resolution=25
2018-08-01 20:31:36,211 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307135126?image=injection_density&resolution=25
2018-08-01 20:32:50,868 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306804565?image=injection_density&resolution=25
2018-08-01 20:35:47,736 allensdk.api.api.retrieve_file_over_

2018-08-01 22:05:44,838 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307163104?image=injection_density&resolution=25
2018-08-01 22:08:11,628 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/307137245?image=injection_density&resolution=25
2018-08-01 22:11:08,164 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/306958740?image=injection_density&resolution=25
2018-08-01 22:12:12,476 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/313387575?image=injection_density&resolution=25
2018-08-01 22:15:37,227 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/310552598?image=injection_density&resolution=25
2018-08-01 22:21:03,776 allensdk.api.api.retrieve_file_over_

### Save & Load station

In [40]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("Master_injection_fraction_dictionary",'wb') as f:
        pickle.dump(Master_injection_fraction_dictionary,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [25]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    Master_injection_fraction_dictionary={}
    with open("Master_injection_fraction_dictionary",'r') as f:
        Master_injection_fraction_dictionary=pickle.load(f)
elif run=="n":
    print "operation cancelled"

Load? (y/n)y


## Reality Check

make sure the fractions add up to no more than 1

#### Filter 


In [26]:
len(Master_injection_fraction_dictionary)

2995

In [27]:
exception_count=0
exception_list=[]

for i in experiment_list['id'].values.tolist():
    for j in Master_injection_fraction_dictionary[i]:
        if Master_injection_fraction_dictionary[i][j]>1:
            raise ValueError("A fraction cannot be larger than 1")


In [28]:
len(exception_list)

0

0 experiments are not valid keys for Master_injection_fraction_dictionary. Investigate

For good reasons, we wish to remove experiments that are too close to one another. First get a map from experiment id to their injection_coordinates

# Get Structure Volume

In [29]:
def volume_dictionary_gen(acr_list):
    """based on input acronym list, return a dictionary that maps ID to its volume.
    
        This is a poor workaround the fact that I don't know how to import the volume data of the structure from the website.
        I can only calculate the volume from the annotation files, but there are discrepencies between that and the unionization
        records.
    
    args:
        acr_list (list): input acronym list
        
    returns:
        returndict(dictionary): map from id to its volume
        exception_structure_acr (list) a list of structure that do not have the volume data available from this method"""
    
    returndict={}
    exception_struct_acr=[]
    
    for i in acr_list:
        
        '''volume data is acquired through unionization.
            From the experiment lists of a structure, I target itself and return its unionization records.
            '''
        struct=structure_tree.get_structures_by_acronym([i])[0]
        struct_experiments=mcc.get_experiments(cre=None,
                                              injection_structure_ids=[struct['id']])
        
        
        '''If error, then it must be the case that the structure does not belong to any injection site.
            I believe it is still possible for other structures to project to these regions, but for this particular analysis,
            I ignore these structures.'''
        try:
            structure_unionizes=mcc.get_structure_unionizes( [e['id'] for e in struct_experiments],
                                                          is_injection=None,
                                                          structure_ids=[struct['id']], # all
                                                          include_descendants=True,
                                                           hemisphere_ids=[2])
        except:
            exception_struct_acr.append(i)
            pass
        
        for j in range(len(structure_unionizes)):
            a=structure_unionizes['structure_id'][j]
            b=structure_unionizes['volume'][j]

            if a not in returndict:
                returndict[a]=b

    return returndict,exception_struct_acr

In [45]:
structure_unionizes=mcc.get_structure_unionizes( [e for e in list(experiment_list['id'])],
                                                          is_injection=None,
                                                          structure_ids=None, # all
                                                          include_descendants=True,
                                                           hemisphere_ids=[2])

C:\Users\Chris OBrien\AppData\Local\conda\conda\envs\p2\lib\site-packages\allensdk\api\cache.py:335: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  data = reader(path)


In [30]:
run=raw_input("computationally demanding. run? (y/n)")

if run=="y":

    id_to_volume={}
    str_id=structure_unionizes['structure_id']
    str_vol=structure_unionizes['volume']
    for i in range(len(str_id)):
        if i not in id_to_volume.keys():
            id_to_volume[str_id[i]]=str_vol[i]
elif run =="n"
    rpint "operation cancelled"

SyntaxError: invalid syntax (<ipython-input-30-09a5d8937d4b>, line 11)

In [31]:
len(id_to_volume)

NameError: name 'id_to_volume' is not defined

In [49]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("id_to_volume",'wb') as f:
        pickle.dump(id_to_volume,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [ ]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    with open("id_to_volume",'r') as f:
        id_to_volume=pickle.load(f)
elif run=="n":
    print "operation cancelled"

In [50]:
id_to_inj_coord=structure_tree

In [51]:
sinj=structure_tree.get_structures_by_acronym(['SCm'])[0]

In [52]:
sinj_experiments=mcc.get_experiments(cre=None,
                                         injection_structure_ids=[sinj['id']])

In [53]:
unionizes=mcc.get_structure_unionizes([ep['id'] for ep in sinj_experiments],
                                         is_injection=False,
                                         structure_ids=[acronym_to_id['AP']],
                                         include_descendants=False,
                                         hemisphere_ids=[2])

In [54]:
unionizes

,hemisphere_id,id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,max_voxel_z,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,experiment_id,structure_id,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
0,2,636051208,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,305025440,207,2.229142e+09,23752800.0,0.000000,0.000000,0.029097
1,2,631773396,False,0.010808,12830,5060,5920,2.933789e-07,0.000006,0.000554,94.313469,1.707713e-07,292533477,207,1.712106e+09,23752800.0,13147.783203,139.405151,0.029097
2,2,634324454,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,307656580,207,4.448998e+09,23752800.0,0.000000,0.000000,0.029097
3,2,636038636,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,305446876,207,1.769696e+08,23752800.0,0.000000,0.000000,0.029097
4,2,631855933,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,294174290,207,1.927530e+08,23752800.0,0.000000,0.000000,0.029097
5,2,629972063,False,0.051576,12760,4930,5840,1.489958e-05,0.000157,0.014939,95.313133,4.560632e-06,175158132,207,2.521453e+09,23752800.0,354847.437500,3722.964844,0.029097
6,2,631919895,False,0.021249,12950,5130,5730,3.502646e-06,0.000011,0.000997,94.791611,3.061630e-07,278511717,207,2.268692e+09,23752800.0,23691.171875,249.928986,0.029097
7,2,630384216,False,0.035377,12400,4850,5820,8.376689e-05,0.000110,0.016814,152.640976,3.205130e-06,182613651,207,8.299907e+08,23752800.0,399374.875000,2616.432861,0.029097
8,2,633672424,False,0.000000,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,126646502,207,1.794206e+09,23752800.0,0.000000,0.000000,0.029097
9,2,633854016,False,0.047275,12800,4980,5920,1.328558e-05,0.000178,0.009861,55.241928,5.193812e-06,146078721,207,9.165370e+08,23752800.0,234217.312500,4239.846680,0.029097


In [55]:
import csv 

expid_to_inj_coord={}
expid_to_inj_volume={}

with open("projection_search_results.csv",'rb') as f:
    full_proj_search=csv.reader(f,delimiter=',')
    
    for row in full_proj_search:
        expid_to_inj_coord[row[0]]=row[14]
        expid_to_inj_volume[row[0]]=row[7]

In [56]:
eic=expid_to_inj_coord

In [57]:
expid_to_injcoord_int={}
for i in eic:
    if eic[i]!='injection-coordinates':
        expid_to_injcoord_int[int(i)]=[int(j) for j in eic[i][1:-1].split(',')]

In [58]:
expid_to_injcoordsquare={}
eic=expid_to_injcoord_int
for i in eic.keys():
    expid_to_injcoordsquare[i]=eic[i][0]**2+eic[i][1]**2+eic[i][2]**2

In [59]:
eics=expid_to_injcoordsquare

In [61]:
expid_to_inj_volume_int={}
for i in expid_to_inj_volume:
    if expid_to_inj_volume[i]!='injection-volume':
        expid_to_inj_volume_int[int(i)]=float(expid_to_inj_volume[i])

inj volume

In [182]:
def sum_proj(inj,target,hemisphere=[2],r1=0.1,r2=1.0):
    """
    args:
        inj (str): acronmy of the injection site
        target (str): acronym of the target structures
    
    returns:
    """
    
    #if not reciprocal:
    #    pass
    #elif reciprocal:
    #    save=inj
    #    inj=target
    #    target=save
        
    #1. get projection search results
    sinj=structure_tree.get_structures_by_acronym([inj])[0]
    starget=structure_tree.get_structures_by_acronym([target])[0]
    
    sinj_experiments=mcc.get_experiments(cre=None,
                                         injection_structure_ids=[sinj['id']])
    
    
    
    if len(sinj_experiments)==0:
        raise ValueError( 'No experiments exist for this structure')
        
    unionizes=mcc.get_structure_unionizes([ep['id'] for ep in sinj_experiments],
                                         is_injection=False,
                                         structure_ids=[starget['id']],
                                         include_descendants=False,
                                         hemisphere_ids=hemisphere)
    exception_count=0.0
    
    #2. using unionized records, get experiments id and its associated projection volume
    # from sinj to target
    a=unionizes['experiment_id']
    b=unionizes['projection_volume']
    
    #3. try fetching the volume data for the injection structure. return error otherwise
    try:
        sinj_structure_vol=float(id_to_volume[acronym_to_id[inj]])/2
    except:
        raise ValueError('Volume data does not exist for this structure')
    
    _error,__error,___error=0.0,0.0,0.0
    
    proj_sum=0.0
    tot_injection_volume_fraction=0.0
    
    for i in range(len(a)):
        p=i
        
        if float(expid_to_inj_volume_int[a[p]])>=r1 and float(expid_to_inj_volume_int[a[p]])<=r2:
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            #c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1
            
            c3=float(expid_to_inj_volume_int[a[p]])
            #c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3
            
    if tot_injection_volume_fraction>=sinj_structure_vol:
        #print "tot inj volume larger"
        #print "tot injection fraction",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        #print "tot inj volume smaller"
        #print "tot inj volume",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
        
        
            
""" 
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1*c2
            _error+=c1*c2*np.sqrt( (0.10)**2 + (0.02)**2)
            
            #tot inj_vol_fraction= injection volume of exp * fraction of the sinj region
            
            c3=float(expid_to_inj_volume_int[a[p]])
            c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3*c4
            __error+=c3*c4*np.sqrt((0.05)**2 + (0.02)**2)
        else:
            pass
    if tot_injection_volume_fraction>=sinj_structure_vol:
        try:
            print "tot inj volume larger"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        except:
            print 'exception occured'
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        try:
            print "tot inj volume smaller"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*sinj_structure_vol/tot_injection_volume_fraction
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        
        except ZeroDivisionError:
            return 0,0,0,0
"""
None

### NOTE: avoid using CTX since its volume is ridicously small


In [382]:
#create list of structures
Cortex_list=[id_to_acronym[k] for k in Substr[acronym_to_id['Isocortex']]]
BS_nuclei=['SCs','SCm','PB','NTS','PAG']
HY_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['HY']]]
TH_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['TH']]]

#### BS to CTX

In [433]:
r1=0.1
r2=1.0

In [409]:
BS_to_CTX={}

for i in BS_nuclei:
    BS_to_CTX[i]={}
    for j in Cortex_list:
        try:
            BS_to_CTX[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [410]:
for i in BS_to_CTX:
    print i,round(sum([BS_to_CTX[i][j] for j in BS_to_CTX[i]]),2)

SCm 71614.62
PB 2288996.35
NTS 332.41
SCs 8981.4
PAG 302999.47


#### CTX to BS

In [434]:
CTX_to_BS={}

for i in Cortex_list:
    CTX_to_BS[i]={}
    for j in BS_nuclei:
        try:
            CTX_to_BS[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

exception PERI SCs
exception PERI SCm
exception PERI PB
exception PERI NTS
exception PERI PAG


In [435]:
CTX_to_BS.pop('PERI',None)

{}

In [446]:
for i in BS_nuclei:
    print round(sum([CTX_to_BS[j][i] for j in CTX_to_BS.keys()]),2)

3885011.58
11517818.81
261102.33
43251.54
4578819.02


In [398]:
for i in CTX_to_BS:
    print i,round(sum([CTX_to_BS[i][j] for j in CTX_to_BS[i]]),2)

VISC 448146.48
SS 1927754.04
ACA 3959033.84
AUD 414137.81
ILA 869132.09
AI 77975.13
TEa 0.0
MO 2649989.21
GU 1539.87
ECT 137089.71
VIS 10282252.17
FRP 10576.86
PTLp 147662.07
ORB 978561.55
RSP 471877.36
PL 238919.97


#### BS to HY

In [437]:
BS_to_HY={}

for i in BS_nuclei:
    BS_to_HY[i]={}
    for j in HY_nuclei:
        try:
            BS_to_HY[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [447]:
for b in BS_nuclei:
    print round(sum(BS_to_HY[b][i] for i in HY_nuclei),2)

2513.23
171857.9
1586207.62
470.78
1283115.73


#### HY to BS

In [439]:
HY_to_BS={}

for i in HY_nuclei:
    HY_to_BS[i]={}
    for j in BS_nuclei:
        try:
            HY_to_BS[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

exception ME SCs
exception ME SCm
exception ME PB
exception ME NTS
exception ME PAG


In [440]:
HY_to_BS.pop('ME',None)

{}

In [448]:
for b in BS_nuclei:
    print round(sum([HY_to_BS[i][b] for i in HY_to_BS]),2)

557274.04
15722071.37
2454173.32
928044.52
41903554.57


#### BS to TH

In [442]:
BS_to_TH={}

for i in BS_nuclei:
    BS_to_TH[i]={}
    for j in TH_nuclei:
        try:
            BS_to_TH[i][j]=sum_proj(i,j,r1=r1,r2=r2)[0]
        except:
            print "exception",i,j

In [449]:
for b in BS_nuclei:
    print round(sum(BS_to_TH[b][i] for i in TH_nuclei),2)

64833.11
601083.29
1132894.97
1115.99
927354.75


#### TH to BS

In [444]:
TH_to_BS={}

for i in TH_nuclei:
    TH_to_BS[i]={}
    for j in BS_nuclei:
        try:
            TH_to_BS[i][j]=sum_proj(i,j,r1=0.1,r2=1.5)[0]
        except:
            print "exception",i,j

In [450]:
for b in BS_nuclei:
    print round(sum([TH_to_BS[i][b] for i in TH_to_BS]),2)

308725.58
1712721.05
70923.13
5445.85
2048788.41


## ILM

In [226]:
sum_proj('ILM','Isocortex',r1=r1,r2=r2)

(6164244.820564197, 'NA')

In [227]:
sum_proj('ILM','TH',r1=r1,r2=r2)

(2580796.7193952, 'NA')

In [228]:
sum_proj('ILM','HY',r1=r1,r2=r2)

(1427266.2691949375, 'NA')

In [229]:
sum_proj('Isocortex','ILM',r1=r1,r2=r2)

(3383596.3011150034, 'NA')

In [230]:
sum_proj('TH','ILM',r1=r1,r2=r2)

(1843627.9539560545, 'NA')

In [231]:
sum_proj('HY','ILM',r1=r1,r2=r2)

(4044288.531278573, 'NA')

In [106]:
_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp','FRP','GU','VISC','PL','ILA','TEa','PERI','ECT']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        summe+=sum_proj(j,i,r1=0.1,r2=1.5)[0]
    print i+' to '+j+" "+str(summe)

SCs to PTLp 61134.6139182


KeyboardInterrupt: 

In [304]:
_regions=['ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        try:
            summe+=sum_proj(j,i,r1=0.1,r2=1.5)[0]
        except:
            None
    print 'HY'+' to '+i+" "+str(summe)

tot inj volume smaller
tot inj volume 0.0
structure vol 0.02743295625
tot inj volume larger
tot injection fraction 0.402797109575
structure vol 0.03433129875
tot inj volume smaller
tot inj volume 0.0
structure vol 0.04993884
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01812951
tot inj volume smaller
tot inj volume 0.0
structure vol 0.030153375
tot inj volume larger
tot injection fraction 3.1599357258
structure vol 0.1531179538
tot inj volume larger
tot injection fraction 2.62867507429
structure vol 0.1036520912
tot inj volume larger
tot injection fraction 0.397480505527
structure vol 0.0520242212
tot inj volume larger
tot injection fraction 0.242690293431
structure vol 0.078535485
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01293949125
tot inj volume larger
tot injection fraction 6.9864844154
structure vol 0.5690300224
tot inj volume larger
tot injection fraction 0.366235996
structure vol 0.154343385
tot inj volume larger
tot injection fraction 0.345275952
s

tot inj volume larger
tot injection fraction 0.242690293431
structure vol 0.078535485
tot inj volume smaller
tot inj volume 0.0
structure vol 0.01293949125
tot inj volume larger
tot injection fraction 6.9864844154
structure vol 0.5690300224
tot inj volume larger
tot injection fraction 0.366235996
structure vol 0.154343385
tot inj volume larger
tot injection fraction 0.345275952
structure vol 0.0458932138
tot inj volume larger
tot injection fraction 0.1261697668
structure vol 0.053636625
tot inj volume larger
tot injection fraction 3.0530966844
structure vol 0.14421141
tot inj volume larger
tot injection fraction 4.3478578276
structure vol 0.4960538912
tot inj volume larger
tot injection fraction 0.199931722331
structure vol 0.03600820125
tot inj volume larger
tot injection fraction 7.25521112633
structure vol 0.1958574688
tot inj volume larger
tot injection fraction 5.02541391624
structure vol 0.27998262635
tot inj volume larger
tot injection fraction 3.30888414999
structure vol 0.1120

In [280]:
sum_proj('BS','HY',r1=0.1,r2=1.5)[0]

81511424.42761406

In [290]:
_regions=['ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']


for i in BS_nuclei:
    summe=0.0
    for j in _regions:
        try:
            summe+=sum_proj(i,j,r1=0.1,r2=1.5)[0]
        except:
            None
    print i+' to '+'HY'+" "+str(summe)

exception occured
exception occured
exception occured
exception occured
exception occured
SCs to HY 2454.48250879
SCm to HY 171806.573731
PB to HY 1585387.81205
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
exception occured
NTS to HY 460.525654608
PAG to HY 1282826.07808
AP to HY 0.0


In [291]:
sum_proj('ILM','CTX',r1=0.1,r2=1.5)[0]

7277884.273278983

In [298]:
sum_proj('CTX','ILM',r1=0.1,r2=1.5)[0]

tot inj volume larger
tot injection fraction 181.077545256
structure vol 0.0430031743938


3842745704.0153522

In [302]:
sum_proj('CTX','SCm',r1=0.1,r2=1.5)[0]

tot inj volume larger
tot injection fraction 181.077545256
structure vol 0.0430031743938


614.7468377566435

In [303]:
614*181/0.043

2584511.6279069767

In [299]:
id_to_volume[acronym_to_id['HY']]

8.3802130048

## Mcmodels
Here we use Mcmodels and their available data to explore the connectivity strength and the information flow


Structure sets used by Knox, et al on 'High resolution data_diven model' uses intermediate level summary structures. This most likely corresponds to the structure set number 687527945 'Structures representing the subdivisions of the Mouse Brain'

In [25]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc=MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
structure_tree=mcc.get_structure_tree()

from mcmodels.core import VoxelModelCache
Voxel_cache = VoxelModelCache()
Voxel_connectivity=Voxel_cache.get_connection_strength()
Voxel_connectivity_dict=dict(Voxel_connectivity)

In [26]:
Voxel_connectivity=Voxel_cache.get_connection_strength()
Voxel_connectivity_dict=dict(Voxel_connectivity)

In [27]:
structure687527945=list(pd.DataFrame(structure_tree.get_structures_by_set_id([687527945]))['acronym'])
Voxel_Structures=set([i[1] for i in Voxel_connectivity_dict.keys()])

In [28]:
print len(structure687527945), len(Voxel_Structures)

293 291


the former has 2 more structures, which are:

In [29]:
for i in structure687527945:
    if i not in Voxel_Structures:
        print i

ENTmv
fiber tracts


In [30]:
for i in Voxel_Structures:
    if i not in structure687527945:
        print i

In [31]:
acronym_to_name['ENTmv']

u'Entorhinal area, medial part, ventral zone'

https://en.wikipedia.org/wiki/Entorhinal_cortex
The entorhinal cortex (EC) (ento = interior, rhino = nose, entorhinal = interior to the rhinal sulcus) is an area of the brain located in the medial temporal lobe and functioning as a hub in a widespread network for memory, navigation and the perception of time.[1] The EC is the main interface between the hippocampus and neocortex

## Exploring Damasio Network
Here we explore the consciousness network proposed by Damasio, which includes the nuclei of the brainstem, hypothalamus, and the cortex.

Since Voxel_Structures only contains structures at the subdivision level, it is not straight forward to get the information flow from large structures to a large structures directly from the Voxel_connectivity. 


We first identify the nuclei of each structures of interest and make sure thoese nuclei are included in Voxel_Structures

Brain stem is divided into:

In [32]:
BSsub=[id_to_name[i] for i in Substr[acronym_to_id['BS']]]
BSsub

[u'Midbrain', u'Hindbrain', u'Interbrain']

where each can further be subdivided into:

In [33]:
MBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[0]]]]
MBsub

[u'Midbrain, motor related',
 u'Midbrain, sensory related',
 u'Midbrain, behavioral state related']

In [34]:
HBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[1]]]]
HBsub

[u'Medulla', u'Pons']

In [35]:
IBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[2]]]]
IBsub

[u'Hypothalamus', u'Thalamus']

but of course non of them are included in the Voxel_Structures, but some of their sub(-sub)structures are definitely included. The question is, which ones? We use tree to visualize this. 

In [36]:
from anytree import Node, RenderTree

In [37]:
import string
def print_code_for_tree(acr):
    """"""
    #returns list of substructures - outputs are in terms of structure id
    substr=[id_to_acronym[i] for i in Substr[acronym_to_id[acr]]]
    
    #variable names cannot contain - and /. replace temporarily
    if "-" in acr or "/" in acr:
        acr=string.replace(acr,"-","_")
        acr=string.replace(acr,"/","and")
    
    if len(substr)>0:
        for j in substr:
            #exp = mcc.get_experiments(file_name='experiments_for_'+j+'.json',
            #                      injection_structure_ids=[acronym_to_id[j]])
            #n_exp=len(exp)
            n_exp='NA'
            
            #check whether the structure is in Voxel_Structures
            if j in Voxel_Structures:
                inVoxel=True
            else:
                inVoxel=False
                    
            #python variable name does not allow - or /. Replace with _ if exists
            if ("-" in j) or ("/" in j):
                
                #check whether the structure is in Voxel_Structures
                j=string.replace(j,"-","_")
                j=string.replace(j,"/","and")

                print str(j)+" = "+"Node('"+ j+" inVoxel="+str(inVoxel) + "  n_exp= "+str(n_exp) +"', parent="+acr+")"
                
                #make sure to return j to its original since otherwise Substr
                #will not recognize its altered form.
                j=string.replace(j,"_","-")
                j=string.replace(j,"and","/")
                
                """RECURSION"""
                print_code_for_tree(j)
                
            else:
                print str(j)+" = "+"Node('"+ j +" inVoxel="+str(inVoxel)+ "  n_exp= "+str(n_exp) +"', parent="+acr+")"
                
                
                """RECURSION"""
                print_code_for_tree(j)
    else:
        None

In [39]:
print_code_for_tree('BS')

MB = Node('MB inVoxel=False  n_exp= NA', parent=BS)
MBmot = Node('MBmot inVoxel=False  n_exp= NA', parent=MB)
III = Node('III inVoxel=True  n_exp= NA', parent=MBmot)
MT = Node('MT inVoxel=False  n_exp= NA', parent=MBmot)
LT = Node('LT inVoxel=True  n_exp= NA', parent=MBmot)
DT = Node('DT inVoxel=False  n_exp= NA', parent=MBmot)
IV = Node('IV inVoxel=True  n_exp= NA', parent=MBmot)
MRN = Node('MRN inVoxel=True  n_exp= NA', parent=MBmot)
PN = Node('PN inVoxel=False  n_exp= NA', parent=MBmot)
RN = Node('RN inVoxel=True  n_exp= NA', parent=MBmot)
AT = Node('AT inVoxel=True  n_exp= NA', parent=MBmot)
RR = Node('RR inVoxel=True  n_exp= NA', parent=MBmot)
SCm = Node('SCm inVoxel=True  n_exp= NA', parent=MBmot)
SCig = Node('SCig inVoxel=False  n_exp= NA', parent=SCm)
SCig_a = Node('SCig_a inVoxel=False  n_exp= NA', parent=SCig)
SCig_b = Node('SCig_b inVoxel=False  n_exp= NA', parent=SCig)
SCig_c = Node('SCig_c inVoxel=False  n_exp= NA', parent=SCig)
SCiw = Node('SCiw inVoxel=False  n_exp= NA',

In [47]:
BS = Node('BS inVoxel=False')
CTX= Node('CTX inVoxel=False')

In [41]:
MB = Node('MB inVoxel=False  n_exp= NA', parent=BS)
MBmot = Node('MBmot inVoxel=False  n_exp= NA', parent=MB)
III = Node('III inVoxel=True  n_exp= NA', parent=MBmot)
MT = Node('MT inVoxel=False  n_exp= NA', parent=MBmot)
LT = Node('LT inVoxel=True  n_exp= NA', parent=MBmot)
DT = Node('DT inVoxel=False  n_exp= NA', parent=MBmot)
IV = Node('IV inVoxel=True  n_exp= NA', parent=MBmot)
MRN = Node('MRN inVoxel=True  n_exp= NA', parent=MBmot)
PN = Node('PN inVoxel=False  n_exp= NA', parent=MBmot)
RN = Node('RN inVoxel=True  n_exp= NA', parent=MBmot)
AT = Node('AT inVoxel=True  n_exp= NA', parent=MBmot)
RR = Node('RR inVoxel=True  n_exp= NA', parent=MBmot)
SCm = Node('SCm inVoxel=True  n_exp= NA', parent=MBmot)
SCig = Node('SCig inVoxel=False  n_exp= NA', parent=SCm)
SCig_a = Node('SCig_a inVoxel=False  n_exp= NA', parent=SCig)
SCig_b = Node('SCig_b inVoxel=False  n_exp= NA', parent=SCig)
SCig_c = Node('SCig_c inVoxel=False  n_exp= NA', parent=SCig)
SCiw = Node('SCiw inVoxel=False  n_exp= NA', parent=SCm)
SCdg = Node('SCdg inVoxel=False  n_exp= NA', parent=SCm)
SCdw = Node('SCdw inVoxel=False  n_exp= NA', parent=SCm)
CUN = Node('CUN inVoxel=True  n_exp= NA', parent=MBmot)
VTA = Node('VTA inVoxel=True  n_exp= NA', parent=MBmot)
VTN = Node('VTN inVoxel=True  n_exp= NA', parent=MBmot)
PAG = Node('PAG inVoxel=True  n_exp= NA', parent=MBmot)
PRC = Node('PRC inVoxel=False  n_exp= NA', parent=PAG)
INC = Node('INC inVoxel=False  n_exp= NA', parent=PAG)
ND = Node('ND inVoxel=False  n_exp= NA', parent=PAG)
Su3 = Node('Su3 inVoxel=False  n_exp= NA', parent=PAG)
EW = Node('EW inVoxel=True  n_exp= NA', parent=MBmot)
Pa4 = Node('Pa4 inVoxel=False  n_exp= NA', parent=MBmot)
PRT = Node('PRT inVoxel=False  n_exp= NA', parent=MBmot)
APN = Node('APN inVoxel=True  n_exp= NA', parent=PRT)
MPT = Node('MPT inVoxel=True  n_exp= NA', parent=PRT)
NOT = Node('NOT inVoxel=True  n_exp= NA', parent=PRT)
NPC = Node('NPC inVoxel=True  n_exp= NA', parent=PRT)
OP = Node('OP inVoxel=True  n_exp= NA', parent=PRT)
PPT = Node('PPT inVoxel=True  n_exp= NA', parent=PRT)
RPF = Node('RPF inVoxel=False  n_exp= NA', parent=PRT)
SNr = Node('SNr inVoxel=True  n_exp= NA', parent=MBmot)
MA3 = Node('MA3 inVoxel=False  n_exp= NA', parent=MBmot)
MBsen = Node('MBsen inVoxel=False  n_exp= NA', parent=MB)
IC = Node('IC inVoxel=True  n_exp= NA', parent=MBsen)
ICc = Node('ICc inVoxel=False  n_exp= NA', parent=IC)
ICd = Node('ICd inVoxel=False  n_exp= NA', parent=IC)
ICe = Node('ICe inVoxel=False  n_exp= NA', parent=IC)
SAG = Node('SAG inVoxel=True  n_exp= NA', parent=MBsen)
SCs = Node('SCs inVoxel=True  n_exp= NA', parent=MBsen)
SCzo = Node('SCzo inVoxel=False  n_exp= NA', parent=SCs)
SCsg = Node('SCsg inVoxel=False  n_exp= NA', parent=SCs)
SCop = Node('SCop inVoxel=False  n_exp= NA', parent=SCs)
MEV = Node('MEV inVoxel=True  n_exp= NA', parent=MBsen)
PBG = Node('PBG inVoxel=True  n_exp= NA', parent=MBsen)
SCO = Node('SCO inVoxel=False  n_exp= NA', parent=MBsen)
NB = Node('NB inVoxel=True  n_exp= NA', parent=MBsen)
MBsta = Node('MBsta inVoxel=False  n_exp= NA', parent=MB)
RAmb = Node('RAmb inVoxel=False  n_exp= NA', parent=MBsta)
IF = Node('IF inVoxel=True  n_exp= NA', parent=RAmb)
IPN = Node('IPN inVoxel=True  n_exp= NA', parent=RAmb)
IPC = Node('IPC inVoxel=False  n_exp= NA', parent=IPN)
IPA = Node('IPA inVoxel=False  n_exp= NA', parent=IPN)
IPL = Node('IPL inVoxel=False  n_exp= NA', parent=IPN)
IPRL = Node('IPRL inVoxel=False  n_exp= NA', parent=IPN)
IPR = Node('IPR inVoxel=False  n_exp= NA', parent=IPN)
IPI = Node('IPI inVoxel=False  n_exp= NA', parent=IPN)
IPDM = Node('IPDM inVoxel=False  n_exp= NA', parent=IPN)
IPDL = Node('IPDL inVoxel=False  n_exp= NA', parent=IPN)
RL = Node('RL inVoxel=True  n_exp= NA', parent=RAmb)
CLI = Node('CLI inVoxel=True  n_exp= NA', parent=RAmb)
DR = Node('DR inVoxel=True  n_exp= NA', parent=RAmb)
SNc = Node('SNc inVoxel=True  n_exp= NA', parent=MBsta)
PPN = Node('PPN inVoxel=True  n_exp= NA', parent=MBsta)
HB = Node('HB inVoxel=False  n_exp= NA', parent=BS)
MY = Node('MY inVoxel=False  n_exp= NA', parent=HB)
MY_mot = Node('MY_mot inVoxel=False  n_exp= NA', parent=MY)
IO = Node('IO inVoxel=True  n_exp= NA', parent=MY_mot)
ISN = Node('ISN inVoxel=True  n_exp= NA', parent=MY_mot)
AMB = Node('AMB inVoxel=True  n_exp= NA', parent=MY_mot)
AMBv = Node('AMBv inVoxel=False  n_exp= NA', parent=AMB)
AMBd = Node('AMBd inVoxel=False  n_exp= NA', parent=AMB)
IRN = Node('IRN inVoxel=True  n_exp= NA', parent=MY_mot)
PHY = Node('PHY inVoxel=False  n_exp= NA', parent=MY_mot)
PRP = Node('PRP inVoxel=True  n_exp= NA', parent=PHY)
NR = Node('NR inVoxel=True  n_exp= NA', parent=PHY)
LIN = Node('LIN inVoxel=True  n_exp= NA', parent=MY_mot)
LRN = Node('LRN inVoxel=True  n_exp= NA', parent=MY_mot)
LRNp = Node('LRNp inVoxel=False  n_exp= NA', parent=LRN)
LRNm = Node('LRNm inVoxel=False  n_exp= NA', parent=LRN)
MARN = Node('MARN inVoxel=True  n_exp= NA', parent=MY_mot)
ICB = Node('ICB inVoxel=True  n_exp= NA', parent=MY_mot)
MDRN = Node('MDRN inVoxel=False  n_exp= NA', parent=MY_mot)
MDRNd = Node('MDRNd inVoxel=True  n_exp= NA', parent=MDRN)
MDRNv = Node('MDRNv inVoxel=True  n_exp= NA', parent=MDRN)
VI = Node('VI inVoxel=True  n_exp= NA', parent=MY_mot)
VII = Node('VII inVoxel=True  n_exp= NA', parent=MY_mot)
x = Node('x inVoxel=True  n_exp= NA', parent=MY_mot)
XII = Node('XII inVoxel=True  n_exp= NA', parent=MY_mot)
y = Node('y inVoxel=True  n_exp= NA', parent=MY_mot)
DMX = Node('DMX inVoxel=True  n_exp= NA', parent=MY_mot)
PARN = Node('PARN inVoxel=True  n_exp= NA', parent=MY_mot)
PAS = Node('PAS inVoxel=True  n_exp= NA', parent=MY_mot)
PGRN = Node('PGRN inVoxel=False  n_exp= NA', parent=MY_mot)
PGRNd = Node('PGRNd inVoxel=True  n_exp= NA', parent=PGRN)
PGRNl = Node('PGRNl inVoxel=True  n_exp= NA', parent=PGRN)
GRN = Node('GRN inVoxel=True  n_exp= NA', parent=MY_mot)
PPY = Node('PPY inVoxel=True  n_exp= NA', parent=MY_mot)
ACVII = Node('ACVII inVoxel=True  n_exp= NA', parent=MY_mot)
VNC = Node('VNC inVoxel=False  n_exp= NA', parent=MY_mot)
LAV = Node('LAV inVoxel=True  n_exp= NA', parent=VNC)
SPIV = Node('SPIV inVoxel=True  n_exp= NA', parent=VNC)
MV = Node('MV inVoxel=True  n_exp= NA', parent=VNC)
SUV = Node('SUV inVoxel=True  n_exp= NA', parent=VNC)
MY_sat = Node('MY_sat inVoxel=False  n_exp= NA', parent=MY)
RPA = Node('RPA inVoxel=True  n_exp= NA', parent=MY_sat)
RM = Node('RM inVoxel=True  n_exp= NA', parent=MY_sat)
RO = Node('RO inVoxel=True  n_exp= NA', parent=MY_sat)
MY_sen = Node('MY_sen inVoxel=False  n_exp= NA', parent=MY)
AP = Node('AP inVoxel=True  n_exp= NA', parent=MY_sen)
SPVI = Node('SPVI inVoxel=True  n_exp= NA', parent=MY_sen)
SPVO = Node('SPVO inVoxel=True  n_exp= NA', parent=MY_sen)
CN = Node('CN inVoxel=False  n_exp= NA', parent=MY_sen)
DCO = Node('DCO inVoxel=True  n_exp= NA', parent=CN)
VCO = Node('VCO inVoxel=True  n_exp= NA', parent=CN)
NTB = Node('NTB inVoxel=True  n_exp= NA', parent=MY_sen)
NTS = Node('NTS inVoxel=True  n_exp= NA', parent=MY_sen)
DCN = Node('DCN inVoxel=False  n_exp= NA', parent=MY_sen)
CU = Node('CU inVoxel=True  n_exp= NA', parent=DCN)
GR = Node('GR inVoxel=True  n_exp= NA', parent=DCN)
ECU = Node('ECU inVoxel=True  n_exp= NA', parent=MY_sen)
SPVC = Node('SPVC inVoxel=True  n_exp= NA', parent=MY_sen)
Pa5 = Node('Pa5 inVoxel=False  n_exp= NA', parent=MY_sen)
P = Node('P inVoxel=False  n_exp= NA', parent=HB)
P_mot = Node('P_mot inVoxel=False  n_exp= NA', parent=P)
B = Node('B inVoxel=True  n_exp= NA', parent=P_mot)
SG = Node('SG inVoxel=True  n_exp= NA', parent=P_mot)
SUT = Node('SUT inVoxel=True  n_exp= NA', parent=P_mot)
TRN = Node('TRN inVoxel=True  n_exp= NA', parent=P_mot)
V = Node('V inVoxel=True  n_exp= NA', parent=P_mot)
DTN = Node('DTN inVoxel=True  n_exp= NA', parent=P_mot)
PCG = Node('PCG inVoxel=True  n_exp= NA', parent=P_mot)
PRNc = Node('PRNc inVoxel=True  n_exp= NA', parent=P_mot)
PDTg = Node('PDTg inVoxel=False  n_exp= NA', parent=P_mot)
PG = Node('PG inVoxel=True  n_exp= NA', parent=P_mot)
P5 = Node('P5 inVoxel=False  n_exp= NA', parent=P_mot)
Acs5 = Node('Acs5 inVoxel=False  n_exp= NA', parent=P_mot)
PC5 = Node('PC5 inVoxel=False  n_exp= NA', parent=P_mot)
I5 = Node('I5 inVoxel=False  n_exp= NA', parent=P_mot)
P_sat = Node('P_sat inVoxel=False  n_exp= NA', parent=P)
PRNr = Node('PRNr inVoxel=True  n_exp= NA', parent=P_sat)
LDT = Node('LDT inVoxel=True  n_exp= NA', parent=P_sat)
RPO = Node('RPO inVoxel=True  n_exp= NA', parent=P_sat)
SLC = Node('SLC inVoxel=True  n_exp= NA', parent=P_sat)
SLD = Node('SLD inVoxel=True  n_exp= NA', parent=P_sat)
NI = Node('NI inVoxel=True  n_exp= NA', parent=P_sat)
CS = Node('CS inVoxel=True  n_exp= NA', parent=P_sat)
LC = Node('LC inVoxel=True  n_exp= NA', parent=P_sat)
P_sen = Node('P_sen inVoxel=False  n_exp= NA', parent=P)
PSV = Node('PSV inVoxel=True  n_exp= NA', parent=P_sen)
SOC = Node('SOC inVoxel=True  n_exp= NA', parent=P_sen)
SOCm = Node('SOCm inVoxel=False  n_exp= NA', parent=SOC)
SOCl = Node('SOCl inVoxel=False  n_exp= NA', parent=SOC)
POR = Node('POR inVoxel=False  n_exp= NA', parent=SOC)
NLL = Node('NLL inVoxel=True  n_exp= NA', parent=P_sen)
PB = Node('PB inVoxel=True  n_exp= NA', parent=P_sen)
KF = Node('KF inVoxel=False  n_exp= NA', parent=PB)
IB = Node('IB inVoxel=False  n_exp= NA', parent=BS)
HY = Node('HY inVoxel=False  n_exp= NA', parent=IB)
PVR = Node('PVR inVoxel=False  n_exp= NA', parent=HY)
ADP = Node('ADP inVoxel=True  n_exp= NA', parent=PVR)
PVp = Node('PVp inVoxel=True  n_exp= NA', parent=PVR)
PVpo = Node('PVpo inVoxel=True  n_exp= NA', parent=PVR)
VLPO = Node('VLPO inVoxel=True  n_exp= NA', parent=PVR)
AVP = Node('AVP inVoxel=True  n_exp= NA', parent=PVR)
AVPV = Node('AVPV inVoxel=True  n_exp= NA', parent=PVR)
SCH = Node('SCH inVoxel=True  n_exp= NA', parent=PVR)
SFO = Node('SFO inVoxel=True  n_exp= NA', parent=PVR)
SBPV = Node('SBPV inVoxel=True  n_exp= NA', parent=PVR)
MEPO = Node('MEPO inVoxel=True  n_exp= NA', parent=PVR)
MPO = Node('MPO inVoxel=True  n_exp= NA', parent=PVR)
OV = Node('OV inVoxel=True  n_exp= NA', parent=PVR)
DMH = Node('DMH inVoxel=True  n_exp= NA', parent=PVR)
PD = Node('PD inVoxel=True  n_exp= NA', parent=PVR)
PS = Node('PS inVoxel=True  n_exp= NA', parent=PVR)
VMPO = Node('VMPO inVoxel=False  n_exp= NA', parent=PVR)
PVZ = Node('PVZ inVoxel=False  n_exp= NA', parent=HY)
PVa = Node('PVa inVoxel=True  n_exp= NA', parent=PVZ)
PVH = Node('PVH inVoxel=True  n_exp= NA', parent=PVZ)
PVi = Node('PVi inVoxel=True  n_exp= NA', parent=PVZ)
ARH = Node('ARH inVoxel=True  n_exp= NA', parent=PVZ)
ASO = Node('ASO inVoxel=True  n_exp= NA', parent=PVZ)
SO = Node('SO inVoxel=True  n_exp= NA', parent=PVZ)
LZ = Node('LZ inVoxel=False  n_exp= NA', parent=HY)
RCH = Node('RCH inVoxel=True  n_exp= NA', parent=LZ)
LHA = Node('LHA inVoxel=True  n_exp= NA', parent=LZ)
LPO = Node('LPO inVoxel=True  n_exp= NA', parent=LZ)
PST = Node('PST inVoxel=True  n_exp= NA', parent=LZ)
PSTN = Node('PSTN inVoxel=True  n_exp= NA', parent=LZ)
STN = Node('STN inVoxel=True  n_exp= NA', parent=LZ)
TU = Node('TU inVoxel=True  n_exp= NA', parent=LZ)
ZI = Node('ZI inVoxel=True  n_exp= NA', parent=LZ)
FF = Node('FF inVoxel=False  n_exp= NA', parent=ZI)
PeF = Node('PeF inVoxel=False  n_exp= NA', parent=LZ)
MEZ = Node('MEZ inVoxel=False  n_exp= NA', parent=HY)
PVHd = Node('PVHd inVoxel=True  n_exp= NA', parent=MEZ)
AHN = Node('AHN inVoxel=True  n_exp= NA', parent=MEZ)
MBO = Node('MBO inVoxel=False  n_exp= NA', parent=MEZ)
LM = Node('LM inVoxel=True  n_exp= NA', parent=MBO)
MM = Node('MM inVoxel=True  n_exp= NA', parent=MBO)
MMme = Node('MMme inVoxel=False  n_exp= NA', parent=MM)
MMm = Node('MMm inVoxel=False  n_exp= NA', parent=MM)
MMd = Node('MMd inVoxel=False  n_exp= NA', parent=MM)
MMl = Node('MMl inVoxel=False  n_exp= NA', parent=MM)
MMp = Node('MMp inVoxel=False  n_exp= NA', parent=MM)
TM = Node('TM inVoxel=False  n_exp= NA', parent=MBO)
TMv = Node('TMv inVoxel=True  n_exp= NA', parent=TM)
TMd = Node('TMd inVoxel=True  n_exp= NA', parent=TM)
SUM = Node('SUM inVoxel=True  n_exp= NA', parent=MBO)
MPN = Node('MPN inVoxel=True  n_exp= NA', parent=MEZ)
VMH = Node('VMH inVoxel=True  n_exp= NA', parent=MEZ)
PH = Node('PH inVoxel=True  n_exp= NA', parent=MEZ)
PMd = Node('PMd inVoxel=True  n_exp= NA', parent=MEZ)
PMv = Node('PMv inVoxel=True  n_exp= NA', parent=MEZ)
ME = Node('ME inVoxel=False  n_exp= NA', parent=HY)
TH = Node('TH inVoxel=False  n_exp= NA', parent=IB)
DORpm = Node('DORpm inVoxel=False  n_exp= NA', parent=TH)
ILM = Node('ILM inVoxel=False  n_exp= NA', parent=DORpm)
RH = Node('RH inVoxel=True  n_exp= NA', parent=ILM)
CL = Node('CL inVoxel=True  n_exp= NA', parent=ILM)
CM = Node('CM inVoxel=True  n_exp= NA', parent=ILM)
PCN = Node('PCN inVoxel=True  n_exp= NA', parent=ILM)
PF = Node('PF inVoxel=True  n_exp= NA', parent=ILM)
PIL = Node('PIL inVoxel=False  n_exp= NA', parent=ILM)
LAT = Node('LAT inVoxel=False  n_exp= NA', parent=DORpm)
LP = Node('LP inVoxel=True  n_exp= NA', parent=LAT)
SGN = Node('SGN inVoxel=True  n_exp= NA', parent=LAT)
PO = Node('PO inVoxel=True  n_exp= NA', parent=LAT)
POL = Node('POL inVoxel=True  n_exp= NA', parent=LAT)
Eth = Node('Eth inVoxel=False  n_exp= NA', parent=LAT)
ATN = Node('ATN inVoxel=False  n_exp= NA', parent=DORpm)
AD = Node('AD inVoxel=True  n_exp= NA', parent=ATN)
AM = Node('AM inVoxel=True  n_exp= NA', parent=ATN)
AMd = Node('AMd inVoxel=False  n_exp= NA', parent=AM)
AMv = Node('AMv inVoxel=False  n_exp= NA', parent=AM)
LD = Node('LD inVoxel=True  n_exp= NA', parent=ATN)
AV = Node('AV inVoxel=True  n_exp= NA', parent=ATN)
IAD = Node('IAD inVoxel=True  n_exp= NA', parent=ATN)
IAM = Node('IAM inVoxel=True  n_exp= NA', parent=ATN)
RT = Node('RT inVoxel=True  n_exp= NA', parent=DORpm)
MED = Node('MED inVoxel=False  n_exp= NA', parent=DORpm)
IMD = Node('IMD inVoxel=True  n_exp= NA', parent=MED)
MD = Node('MD inVoxel=True  n_exp= NA', parent=MED)
SMT = Node('SMT inVoxel=True  n_exp= NA', parent=MED)
PR = Node('PR inVoxel=True  n_exp= NA', parent=MED)
MTN = Node('MTN inVoxel=False  n_exp= NA', parent=DORpm)
PT = Node('PT inVoxel=True  n_exp= NA', parent=MTN)
PVT = Node('PVT inVoxel=True  n_exp= NA', parent=MTN)
RE = Node('RE inVoxel=True  n_exp= NA', parent=MTN)
Xi = Node('Xi inVoxel=False  n_exp= NA', parent=MTN)
EPI = Node('EPI inVoxel=False  n_exp= NA', parent=DORpm)
LH = Node('LH inVoxel=True  n_exp= NA', parent=EPI)
MH = Node('MH inVoxel=True  n_exp= NA', parent=EPI)
GENv = Node('GENv inVoxel=False  n_exp= NA', parent=DORpm)
IGL = Node('IGL inVoxel=True  n_exp= NA', parent=GENv)
LGv = Node('LGv inVoxel=True  n_exp= NA', parent=GENv)
SubG = Node('SubG inVoxel=True  n_exp= NA', parent=GENv)
IntG = Node('IntG inVoxel=False  n_exp= NA', parent=GENv)
DORsm = Node('DORsm inVoxel=False  n_exp= NA', parent=TH)
SPF = Node('SPF inVoxel=False  n_exp= NA', parent=DORsm)
SPFm = Node('SPFm inVoxel=True  n_exp= NA', parent=SPF)
SPFp = Node('SPFp inVoxel=True  n_exp= NA', parent=SPF)
SPA = Node('SPA inVoxel=True  n_exp= NA', parent=DORsm)
VENT = Node('VENT inVoxel=False  n_exp= NA', parent=DORsm)
VM = Node('VM inVoxel=True  n_exp= NA', parent=VENT)
PoT = Node('PoT inVoxel=False  n_exp= NA', parent=VENT)
VAL = Node('VAL inVoxel=True  n_exp= NA', parent=VENT)
VP = Node('VP inVoxel=False  n_exp= NA', parent=VENT)
VPL = Node('VPL inVoxel=True  n_exp= NA', parent=VP)
VPLpc = Node('VPLpc inVoxel=True  n_exp= NA', parent=VP)
VPM = Node('VPM inVoxel=True  n_exp= NA', parent=VP)
VPMpc = Node('VPMpc inVoxel=True  n_exp= NA', parent=VP)
GENd = Node('GENd inVoxel=False  n_exp= NA', parent=DORsm)
LGd = Node('LGd inVoxel=True  n_exp= NA', parent=GENd)
LGd_sh = Node('LGd_sh inVoxel=False  n_exp= NA', parent=LGd)
LGd_co = Node('LGd_co inVoxel=False  n_exp= NA', parent=LGd)
LGd_ip = Node('LGd_ip inVoxel=False  n_exp= NA', parent=LGd)
MG = Node('MG inVoxel=True  n_exp= NA', parent=GENd)
MGd = Node('MGd inVoxel=False  n_exp= NA', parent=MG)
MGv = Node('MGv inVoxel=False  n_exp= NA', parent=MG)
MGm = Node('MGm inVoxel=False  n_exp= NA', parent=MG)
PP = Node('PP inVoxel=True  n_exp= NA', parent=DORsm)

In [42]:
for pre, fill, node in RenderTree(BS):
    print   pre,node.name

 BS inVoxel=False
├──  MB inVoxel=False  n_exp= NA
│   ├──  MBmot inVoxel=False  n_exp= NA
│   │   ├──  III inVoxel=True  n_exp= NA
│   │   ├──  MT inVoxel=False  n_exp= NA
│   │   ├──  LT inVoxel=True  n_exp= NA
│   │   ├──  DT inVoxel=False  n_exp= NA
│   │   ├──  IV inVoxel=True  n_exp= NA
│   │   ├──  MRN inVoxel=True  n_exp= NA
│   │   ├──  PN inVoxel=False  n_exp= NA
│   │   ├──  RN inVoxel=True  n_exp= NA
│   │   ├──  AT inVoxel=True  n_exp= NA
│   │   ├──  RR inVoxel=True  n_exp= NA
│   │   ├──  SCm inVoxel=True  n_exp= NA
│   │   │   ├──  SCig inVoxel=False  n_exp= NA
│   │   │   │   ├──  SCig_a inVoxel=False  n_exp= NA
│   │   │   │   ├──  SCig_b inVoxel=False  n_exp= NA
│   │   │   │   └──  SCig_c inVoxel=False  n_exp= NA
│   │   │   ├──  SCiw inVoxel=False  n_exp= NA
│   │   │   ├──  SCdg inVoxel=False  n_exp= NA
│   │   │   └──  SCdw inVoxel=False  n_exp= NA
│   │   ├──  CUN inVoxel=True  n_exp= NA
│   │   ├──  VTA inVoxel=True  n_exp= NA
│   │   ├──  VTN inVoxel=True  n_e

In [55]:
print_code_for_tree('CTX')

CTXpl = Node('CTXpl inVoxel=False  n_exp= NA', parent=CTX)
Isocortex = Node('Isocortex inVoxel=False  n_exp= NA', parent=CTXpl)
PTLp = Node('PTLp inVoxel=False  n_exp= NA', parent=Isocortex)
VISa = Node('VISa inVoxel=True  n_exp= NA', parent=PTLp)
VISa1 = Node('VISa1 inVoxel=False  n_exp= NA', parent=VISa)
VISa2and3 = Node('VISa2and3 inVoxel=False  n_exp= NA', parent=VISa)
VISa4 = Node('VISa4 inVoxel=False  n_exp= NA', parent=VISa)
VISa5 = Node('VISa5 inVoxel=False  n_exp= NA', parent=VISa)
VISa6a = Node('VISa6a inVoxel=False  n_exp= NA', parent=VISa)
VISa6b = Node('VISa6b inVoxel=False  n_exp= NA', parent=VISa)
VISrl = Node('VISrl inVoxel=True  n_exp= NA', parent=PTLp)
VISrl1 = Node('VISrl1 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl2and3 = Node('VISrl2and3 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl4 = Node('VISrl4 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl5 = Node('VISrl5 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6a = Node('VISrl6a inVoxel=False  n_exp= NA', parent=VIS

In [56]:
CTXpl = Node('CTXpl inVoxel=False  n_exp= NA', parent=CTX)
Isocortex = Node('Isocortex inVoxel=False  n_exp= NA', parent=CTXpl)
PTLp = Node('PTLp inVoxel=False  n_exp= NA', parent=Isocortex)
VISa = Node('VISa inVoxel=True  n_exp= NA', parent=PTLp)
VISa1 = Node('VISa1 inVoxel=False  n_exp= NA', parent=VISa)
VISa2and3 = Node('VISa2and3 inVoxel=False  n_exp= NA', parent=VISa)
VISa4 = Node('VISa4 inVoxel=False  n_exp= NA', parent=VISa)
VISa5 = Node('VISa5 inVoxel=False  n_exp= NA', parent=VISa)
VISa6a = Node('VISa6a inVoxel=False  n_exp= NA', parent=VISa)
VISa6b = Node('VISa6b inVoxel=False  n_exp= NA', parent=VISa)
VISrl = Node('VISrl inVoxel=True  n_exp= NA', parent=PTLp)
VISrl1 = Node('VISrl1 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl2and3 = Node('VISrl2and3 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl4 = Node('VISrl4 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl5 = Node('VISrl5 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6a = Node('VISrl6a inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6b = Node('VISrl6b inVoxel=False  n_exp= NA', parent=VISrl)
ACA = Node('ACA inVoxel=False  n_exp= NA', parent=Isocortex)
ACAd = Node('ACAd inVoxel=True  n_exp= NA', parent=ACA)
ACAd2and3 = Node('ACAd2and3 inVoxel=False  n_exp= NA', parent=ACAd)
ACAd6a = Node('ACAd6a inVoxel=False  n_exp= NA', parent=ACAd)
ACAd6b = Node('ACAd6b inVoxel=False  n_exp= NA', parent=ACAd)
ACAd1 = Node('ACAd1 inVoxel=False  n_exp= NA', parent=ACAd)
ACAd5 = Node('ACAd5 inVoxel=False  n_exp= NA', parent=ACAd)
ACAv = Node('ACAv inVoxel=True  n_exp= NA', parent=ACA)
ACAv2and3 = Node('ACAv2and3 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv1 = Node('ACAv1 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv5 = Node('ACAv5 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv6a = Node('ACAv6a inVoxel=False  n_exp= NA', parent=ACAv)
ACAv6b = Node('ACAv6b inVoxel=False  n_exp= NA', parent=ACAv)
ILA = Node('ILA inVoxel=True  n_exp= NA', parent=Isocortex)
ILA2and3 = Node('ILA2and3 inVoxel=False  n_exp= NA', parent=ILA)
ILA1 = Node('ILA1 inVoxel=False  n_exp= NA', parent=ILA)
ILA5 = Node('ILA5 inVoxel=False  n_exp= NA', parent=ILA)
ILA6a = Node('ILA6a inVoxel=False  n_exp= NA', parent=ILA)
ILA6b = Node('ILA6b inVoxel=False  n_exp= NA', parent=ILA)
AI = Node('AI inVoxel=False  n_exp= NA', parent=Isocortex)
AId = Node('AId inVoxel=True  n_exp= NA', parent=AI)
AId2and3 = Node('AId2and3 inVoxel=False  n_exp= NA', parent=AId)
AId6a = Node('AId6a inVoxel=False  n_exp= NA', parent=AId)
AId6b = Node('AId6b inVoxel=False  n_exp= NA', parent=AId)
AId1 = Node('AId1 inVoxel=False  n_exp= NA', parent=AId)
AId5 = Node('AId5 inVoxel=False  n_exp= NA', parent=AId)
AIp = Node('AIp inVoxel=True  n_exp= NA', parent=AI)
AIp1 = Node('AIp1 inVoxel=False  n_exp= NA', parent=AIp)
AIp2and3 = Node('AIp2and3 inVoxel=False  n_exp= NA', parent=AIp)
AIp6a = Node('AIp6a inVoxel=False  n_exp= NA', parent=AIp)
AIp5 = Node('AIp5 inVoxel=False  n_exp= NA', parent=AIp)
AIp6b = Node('AIp6b inVoxel=False  n_exp= NA', parent=AIp)
AIv = Node('AIv inVoxel=True  n_exp= NA', parent=AI)
AIv6a = Node('AIv6a inVoxel=False  n_exp= NA', parent=AIv)
AIv2and3 = Node('AIv2and3 inVoxel=False  n_exp= NA', parent=AIv)
AIv6b = Node('AIv6b inVoxel=False  n_exp= NA', parent=AIv)
AIv1 = Node('AIv1 inVoxel=False  n_exp= NA', parent=AIv)
AIv5 = Node('AIv5 inVoxel=False  n_exp= NA', parent=AIv)
FRP = Node('FRP inVoxel=True  n_exp= NA', parent=Isocortex)
FRP1 = Node('FRP1 inVoxel=False  n_exp= NA', parent=FRP)
FRP2and3 = Node('FRP2and3 inVoxel=False  n_exp= NA', parent=FRP)
FRP5 = Node('FRP5 inVoxel=False  n_exp= NA', parent=FRP)
FRP6a = Node('FRP6a inVoxel=False  n_exp= NA', parent=FRP)
FRP6b = Node('FRP6b inVoxel=False  n_exp= NA', parent=FRP)
AUD = Node('AUD inVoxel=False  n_exp= NA', parent=Isocortex)
AUDp = Node('AUDp inVoxel=True  n_exp= NA', parent=AUD)
AUDp2and3 = Node('AUDp2and3 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp1 = Node('AUDp1 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp4 = Node('AUDp4 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp5 = Node('AUDp5 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp6a = Node('AUDp6a inVoxel=False  n_exp= NA', parent=AUDp)
AUDp6b = Node('AUDp6b inVoxel=False  n_exp= NA', parent=AUDp)
AUDd = Node('AUDd inVoxel=True  n_exp= NA', parent=AUD)
AUDd6a = Node('AUDd6a inVoxel=False  n_exp= NA', parent=AUDd)
AUDd6b = Node('AUDd6b inVoxel=False  n_exp= NA', parent=AUDd)
AUDd5 = Node('AUDd5 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd1 = Node('AUDd1 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd2and3 = Node('AUDd2and3 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd4 = Node('AUDd4 inVoxel=False  n_exp= NA', parent=AUDd)
AUDv = Node('AUDv inVoxel=True  n_exp= NA', parent=AUD)
AUDv6a = Node('AUDv6a inVoxel=False  n_exp= NA', parent=AUDv)
AUDv6b = Node('AUDv6b inVoxel=False  n_exp= NA', parent=AUDv)
AUDv2and3 = Node('AUDv2and3 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv1 = Node('AUDv1 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv4 = Node('AUDv4 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv5 = Node('AUDv5 inVoxel=False  n_exp= NA', parent=AUDv)
AUDpo = Node('AUDpo inVoxel=True  n_exp= NA', parent=AUD)
AUDpo6a = Node('AUDpo6a inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo6b = Node('AUDpo6b inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo2and3 = Node('AUDpo2and3 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo1 = Node('AUDpo1 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo4 = Node('AUDpo4 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo5 = Node('AUDpo5 inVoxel=False  n_exp= NA', parent=AUDpo)
RSP = Node('RSP inVoxel=False  n_exp= NA', parent=Isocortex)
RSPd = Node('RSPd inVoxel=True  n_exp= NA', parent=RSP)
RSPd6a = Node('RSPd6a inVoxel=False  n_exp= NA', parent=RSPd)
RSPd6b = Node('RSPd6b inVoxel=False  n_exp= NA', parent=RSPd)
RSPd2and3 = Node('RSPd2and3 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd1 = Node('RSPd1 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd4 = Node('RSPd4 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd5 = Node('RSPd5 inVoxel=False  n_exp= NA', parent=RSPd)
RSPv = Node('RSPv inVoxel=True  n_exp= NA', parent=RSP)
RSPv2and3 = Node('RSPv2and3 inVoxel=False  n_exp= NA', parent=RSPv)
RSPv1 = Node('RSPv1 inVoxel=False  n_exp= NA', parent=RSPv)
RSPv6a = Node('RSPv6a inVoxel=False  n_exp= NA', parent=RSPv)
RSPv6b = Node('RSPv6b inVoxel=False  n_exp= NA', parent=RSPv)
RSPv5 = Node('RSPv5 inVoxel=False  n_exp= NA', parent=RSPv)
RSPagl = Node('RSPagl inVoxel=True  n_exp= NA', parent=RSP)
RSPagl6b = Node('RSPagl6b inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl1 = Node('RSPagl1 inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl5 = Node('RSPagl5 inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl6a = Node('RSPagl6a inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl2and3 = Node('RSPagl2and3 inVoxel=False  n_exp= NA', parent=RSPagl)
MO = Node('MO inVoxel=False  n_exp= NA', parent=Isocortex)
MOp = Node('MOp inVoxel=True  n_exp= NA', parent=MO)
MOp1 = Node('MOp1 inVoxel=False  n_exp= NA', parent=MOp)
MOp5 = Node('MOp5 inVoxel=False  n_exp= NA', parent=MOp)
MOp6a = Node('MOp6a inVoxel=False  n_exp= NA', parent=MOp)
MOp6b = Node('MOp6b inVoxel=False  n_exp= NA', parent=MOp)
MOp2and3 = Node('MOp2and3 inVoxel=False  n_exp= NA', parent=MOp)
MOs = Node('MOs inVoxel=True  n_exp= NA', parent=MO)
MOs1 = Node('MOs1 inVoxel=False  n_exp= NA', parent=MOs)
MOs5 = Node('MOs5 inVoxel=False  n_exp= NA', parent=MOs)
MOs2and3 = Node('MOs2and3 inVoxel=False  n_exp= NA', parent=MOs)
MOs6a = Node('MOs6a inVoxel=False  n_exp= NA', parent=MOs)
MOs6b = Node('MOs6b inVoxel=False  n_exp= NA', parent=MOs)
TEa = Node('TEa inVoxel=True  n_exp= NA', parent=Isocortex)
TEa1 = Node('TEa1 inVoxel=False  n_exp= NA', parent=TEa)
TEa4 = Node('TEa4 inVoxel=False  n_exp= NA', parent=TEa)
TEa5 = Node('TEa5 inVoxel=False  n_exp= NA', parent=TEa)
TEa6a = Node('TEa6a inVoxel=False  n_exp= NA', parent=TEa)
TEa6b = Node('TEa6b inVoxel=False  n_exp= NA', parent=TEa)
TEa2and3 = Node('TEa2and3 inVoxel=False  n_exp= NA', parent=TEa)
VIS = Node('VIS inVoxel=False  n_exp= NA', parent=Isocortex)
VISp = Node('VISp inVoxel=True  n_exp= NA', parent=VIS)
VISp6a = Node('VISp6a inVoxel=False  n_exp= NA', parent=VISp)
VISp6b = Node('VISp6b inVoxel=False  n_exp= NA', parent=VISp)
VISp1 = Node('VISp1 inVoxel=False  n_exp= NA', parent=VISp)
VISp4 = Node('VISp4 inVoxel=False  n_exp= NA', parent=VISp)
VISp5 = Node('VISp5 inVoxel=False  n_exp= NA', parent=VISp)
VISp2and3 = Node('VISp2and3 inVoxel=False  n_exp= NA', parent=VISp)
VISam = Node('VISam inVoxel=True  n_exp= NA', parent=VIS)
VISam1 = Node('VISam1 inVoxel=False  n_exp= NA', parent=VISam)
VISam4 = Node('VISam4 inVoxel=False  n_exp= NA', parent=VISam)
VISam5 = Node('VISam5 inVoxel=False  n_exp= NA', parent=VISam)
VISam6b = Node('VISam6b inVoxel=False  n_exp= NA', parent=VISam)
VISam6a = Node('VISam6a inVoxel=False  n_exp= NA', parent=VISam)
VISam2and3 = Node('VISam2and3 inVoxel=False  n_exp= NA', parent=VISam)
VISal = Node('VISal inVoxel=True  n_exp= NA', parent=VIS)
VISal5 = Node('VISal5 inVoxel=False  n_exp= NA', parent=VISal)
VISal6a = Node('VISal6a inVoxel=False  n_exp= NA', parent=VISal)
VISal6b = Node('VISal6b inVoxel=False  n_exp= NA', parent=VISal)
VISal2and3 = Node('VISal2and3 inVoxel=False  n_exp= NA', parent=VISal)
VISal1 = Node('VISal1 inVoxel=False  n_exp= NA', parent=VISal)
VISal4 = Node('VISal4 inVoxel=False  n_exp= NA', parent=VISal)
VISli = Node('VISli inVoxel=True  n_exp= NA', parent=VIS)
VISli2and3 = Node('VISli2and3 inVoxel=False  n_exp= NA', parent=VISli)
VISli4 = Node('VISli4 inVoxel=False  n_exp= NA', parent=VISli)
VISli5 = Node('VISli5 inVoxel=False  n_exp= NA', parent=VISli)
VISli1 = Node('VISli1 inVoxel=False  n_exp= NA', parent=VISli)
VISli6a = Node('VISli6a inVoxel=False  n_exp= NA', parent=VISli)
VISli6b = Node('VISli6b inVoxel=False  n_exp= NA', parent=VISli)
VISl = Node('VISl inVoxel=True  n_exp= NA', parent=VIS)
VISl6a = Node('VISl6a inVoxel=False  n_exp= NA', parent=VISl)
VISl6b = Node('VISl6b inVoxel=False  n_exp= NA', parent=VISl)
VISl1 = Node('VISl1 inVoxel=False  n_exp= NA', parent=VISl)
VISl4 = Node('VISl4 inVoxel=False  n_exp= NA', parent=VISl)
VISl5 = Node('VISl5 inVoxel=False  n_exp= NA', parent=VISl)
VISl2and3 = Node('VISl2and3 inVoxel=False  n_exp= NA', parent=VISl)
VISpl = Node('VISpl inVoxel=True  n_exp= NA', parent=VIS)
VISpl2and3 = Node('VISpl2and3 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl6a = Node('VISpl6a inVoxel=False  n_exp= NA', parent=VISpl)
VISpl6b = Node('VISpl6b inVoxel=False  n_exp= NA', parent=VISpl)
VISpl1 = Node('VISpl1 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl4 = Node('VISpl4 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl5 = Node('VISpl5 inVoxel=False  n_exp= NA', parent=VISpl)
VISpm = Node('VISpm inVoxel=True  n_exp= NA', parent=VIS)
VISpm2and3 = Node('VISpm2and3 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm6a = Node('VISpm6a inVoxel=False  n_exp= NA', parent=VISpm)
VISpm6b = Node('VISpm6b inVoxel=False  n_exp= NA', parent=VISpm)
VISpm5 = Node('VISpm5 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm4 = Node('VISpm4 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm1 = Node('VISpm1 inVoxel=False  n_exp= NA', parent=VISpm)
VISpor = Node('VISpor inVoxel=True  n_exp= NA', parent=VIS)
VISpor1 = Node('VISpor1 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor2and3 = Node('VISpor2and3 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor4 = Node('VISpor4 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor5 = Node('VISpor5 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor6a = Node('VISpor6a inVoxel=False  n_exp= NA', parent=VISpor)
VISpor6b = Node('VISpor6b inVoxel=False  n_exp= NA', parent=VISpor)
VISC = Node('VISC inVoxel=True  n_exp= NA', parent=Isocortex)
VISC6b = Node('VISC6b inVoxel=False  n_exp= NA', parent=VISC)
VISC6a = Node('VISC6a inVoxel=False  n_exp= NA', parent=VISC)
VISC1 = Node('VISC1 inVoxel=False  n_exp= NA', parent=VISC)
VISC4 = Node('VISC4 inVoxel=False  n_exp= NA', parent=VISC)
VISC5 = Node('VISC5 inVoxel=False  n_exp= NA', parent=VISC)
VISC2and3 = Node('VISC2and3 inVoxel=False  n_exp= NA', parent=VISC)
ORB = Node('ORB inVoxel=False  n_exp= NA', parent=Isocortex)
ORBl = Node('ORBl inVoxel=True  n_exp= NA', parent=ORB)
ORBl2and3 = Node('ORBl2and3 inVoxel=False  n_exp= NA', parent=ORBl)
ORBl6a = Node('ORBl6a inVoxel=False  n_exp= NA', parent=ORBl)
ORBl1 = Node('ORBl1 inVoxel=False  n_exp= NA', parent=ORBl)
ORBl6b = Node('ORBl6b inVoxel=False  n_exp= NA', parent=ORBl)
ORBl5 = Node('ORBl5 inVoxel=False  n_exp= NA', parent=ORBl)
ORBm = Node('ORBm inVoxel=True  n_exp= NA', parent=ORB)
ORBm1 = Node('ORBm1 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm2and3 = Node('ORBm2and3 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm5 = Node('ORBm5 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm6a = Node('ORBm6a inVoxel=False  n_exp= NA', parent=ORBm)
ORBm6b = Node('ORBm6b inVoxel=False  n_exp= NA', parent=ORBm)
ORBvl = Node('ORBvl inVoxel=True  n_exp= NA', parent=ORB)
ORBvl2and3 = Node('ORBvl2and3 inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl6a = Node('ORBvl6a inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl6b = Node('ORBvl6b inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl1 = Node('ORBvl1 inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl5 = Node('ORBvl5 inVoxel=False  n_exp= NA', parent=ORBvl)
ECT = Node('ECT inVoxel=True  n_exp= NA', parent=Isocortex)
ECT2and3 = Node('ECT2and3 inVoxel=False  n_exp= NA', parent=ECT)
ECT1 = Node('ECT1 inVoxel=False  n_exp= NA', parent=ECT)
ECT6a = Node('ECT6a inVoxel=False  n_exp= NA', parent=ECT)
ECT5 = Node('ECT5 inVoxel=False  n_exp= NA', parent=ECT)
ECT6b = Node('ECT6b inVoxel=False  n_exp= NA', parent=ECT)
PERI = Node('PERI inVoxel=True  n_exp= NA', parent=Isocortex)
PERI6a = Node('PERI6a inVoxel=False  n_exp= NA', parent=PERI)
PERI6b = Node('PERI6b inVoxel=False  n_exp= NA', parent=PERI)
PERI1 = Node('PERI1 inVoxel=False  n_exp= NA', parent=PERI)
PERI5 = Node('PERI5 inVoxel=False  n_exp= NA', parent=PERI)
PERI2and3 = Node('PERI2and3 inVoxel=False  n_exp= NA', parent=PERI)
PL = Node('PL inVoxel=True  n_exp= NA', parent=Isocortex)
PL6a = Node('PL6a inVoxel=False  n_exp= NA', parent=PL)
PL6b = Node('PL6b inVoxel=False  n_exp= NA', parent=PL)
PL1 = Node('PL1 inVoxel=False  n_exp= NA', parent=PL)
PL2and3 = Node('PL2and3 inVoxel=False  n_exp= NA', parent=PL)
PL5 = Node('PL5 inVoxel=False  n_exp= NA', parent=PL)
GU = Node('GU inVoxel=True  n_exp= NA', parent=Isocortex)
GU1 = Node('GU1 inVoxel=False  n_exp= NA', parent=GU)
GU4 = Node('GU4 inVoxel=False  n_exp= NA', parent=GU)
GU2and3 = Node('GU2and3 inVoxel=False  n_exp= NA', parent=GU)
GU5 = Node('GU5 inVoxel=False  n_exp= NA', parent=GU)
GU6a = Node('GU6a inVoxel=False  n_exp= NA', parent=GU)
GU6b = Node('GU6b inVoxel=False  n_exp= NA', parent=GU)
SS = Node('SS inVoxel=False  n_exp= NA', parent=Isocortex)
SSp = Node('SSp inVoxel=False  n_exp= NA', parent=SS)
SSp_bfd = Node('SSp_bfd inVoxel=True  n_exp= NA', parent=SSp)
SSp_bfd2and3 = Node('SSp_bfd2and3 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd1 = Node('SSp_bfd1 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd6a = Node('SSp_bfd6a inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd4 = Node('SSp_bfd4 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd6b = Node('SSp_bfd6b inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd5 = Node('SSp_bfd5 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_ll = Node('SSp_ll inVoxel=True  n_exp= NA', parent=SSp)
SSp_ll2and3 = Node('SSp_ll2and3 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll6a = Node('SSp_ll6a inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll6b = Node('SSp_ll6b inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll1 = Node('SSp_ll1 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll4 = Node('SSp_ll4 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll5 = Node('SSp_ll5 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_m = Node('SSp_m inVoxel=True  n_exp= NA', parent=SSp)
SSp_m6b = Node('SSp_m6b inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m2and3 = Node('SSp_m2and3 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m1 = Node('SSp_m1 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m4 = Node('SSp_m4 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m5 = Node('SSp_m5 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m6a = Node('SSp_m6a inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_n = Node('SSp_n inVoxel=True  n_exp= NA', parent=SSp)
SSp_n1 = Node('SSp_n1 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n4 = Node('SSp_n4 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n5 = Node('SSp_n5 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n2and3 = Node('SSp_n2and3 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n6a = Node('SSp_n6a inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n6b = Node('SSp_n6b inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_tr = Node('SSp_tr inVoxel=True  n_exp= NA', parent=SSp)
SSp_tr6a = Node('SSp_tr6a inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr6b = Node('SSp_tr6b inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr2and3 = Node('SSp_tr2and3 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr1 = Node('SSp_tr1 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr4 = Node('SSp_tr4 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr5 = Node('SSp_tr5 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_ul = Node('SSp_ul inVoxel=True  n_exp= NA', parent=SSp)
SSp_ul1 = Node('SSp_ul1 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul4 = Node('SSp_ul4 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul5 = Node('SSp_ul5 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul2and3 = Node('SSp_ul2and3 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul6b = Node('SSp_ul6b inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul6a = Node('SSp_ul6a inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_un = Node('SSp_un inVoxel=True  n_exp= NA', parent=SSp)
SSp_un1 = Node('SSp_un1 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un4 = Node('SSp_un4 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un5 = Node('SSp_un5 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un6a = Node('SSp_un6a inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un6b = Node('SSp_un6b inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un2and3 = Node('SSp_un2and3 inVoxel=False  n_exp= NA', parent=SSp_un)
SSs = Node('SSs inVoxel=True  n_exp= NA', parent=SS)
SSs2and3 = Node('SSs2and3 inVoxel=False  n_exp= NA', parent=SSs)
SSs6a = Node('SSs6a inVoxel=False  n_exp= NA', parent=SSs)
SSs1 = Node('SSs1 inVoxel=False  n_exp= NA', parent=SSs)
SSs6b = Node('SSs6b inVoxel=False  n_exp= NA', parent=SSs)
SSs4 = Node('SSs4 inVoxel=False  n_exp= NA', parent=SSs)
SSs5 = Node('SSs5 inVoxel=False  n_exp= NA', parent=SSs)
OLF = Node('OLF inVoxel=False  n_exp= NA', parent=CTXpl)
AOB = Node('AOB inVoxel=True  n_exp= NA', parent=OLF)
AOBgl = Node('AOBgl inVoxel=False  n_exp= NA', parent=AOB)
AOBgr = Node('AOBgr inVoxel=False  n_exp= NA', parent=AOB)
AOBmi = Node('AOBmi inVoxel=False  n_exp= NA', parent=AOB)
AON = Node('AON inVoxel=True  n_exp= NA', parent=OLF)
MOB = Node('MOB inVoxel=True  n_exp= NA', parent=OLF)
TR = Node('TR inVoxel=True  n_exp= NA', parent=OLF)
TT = Node('TT inVoxel=True  n_exp= NA', parent=OLF)
TTd = Node('TTd inVoxel=False  n_exp= NA', parent=TT)
TTv = Node('TTv inVoxel=False  n_exp= NA', parent=TT)
NLOT = Node('NLOT inVoxel=True  n_exp= NA', parent=OLF)
NLOT1 = Node('NLOT1 inVoxel=False  n_exp= NA', parent=NLOT)
NLOT2 = Node('NLOT2 inVoxel=False  n_exp= NA', parent=NLOT)
NLOT3 = Node('NLOT3 inVoxel=False  n_exp= NA', parent=NLOT)
COA = Node('COA inVoxel=False  n_exp= NA', parent=OLF)
COAa = Node('COAa inVoxel=True  n_exp= NA', parent=COA)
COAp = Node('COAp inVoxel=True  n_exp= NA', parent=COA)
COApl = Node('COApl inVoxel=False  n_exp= NA', parent=COAp)
COApm = Node('COApm inVoxel=False  n_exp= NA', parent=COAp)
PAA = Node('PAA inVoxel=True  n_exp= NA', parent=OLF)
DP = Node('DP inVoxel=True  n_exp= NA', parent=OLF)
PIR = Node('PIR inVoxel=True  n_exp= NA', parent=OLF)
HPF = Node('HPF inVoxel=False  n_exp= NA', parent=CTXpl)
RHP = Node('RHP inVoxel=False  n_exp= NA', parent=HPF)
SUB = Node('SUB inVoxel=True  n_exp= NA', parent=RHP)
APr = Node('APr inVoxel=False  n_exp= NA', parent=RHP)
PAR = Node('PAR inVoxel=True  n_exp= NA', parent=RHP)
ENT = Node('ENT inVoxel=False  n_exp= NA', parent=RHP)
ENTl = Node('ENTl inVoxel=True  n_exp= NA', parent=ENT)
ENTl2 = Node('ENTl2 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl6a = Node('ENTl6a inVoxel=False  n_exp= NA', parent=ENTl)
ENTl3 = Node('ENTl3 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl5 = Node('ENTl5 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl1 = Node('ENTl1 inVoxel=False  n_exp= NA', parent=ENTl)
ENTm = Node('ENTm inVoxel=True  n_exp= NA', parent=ENT)
ENTm1 = Node('ENTm1 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm2 = Node('ENTm2 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm3 = Node('ENTm3 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm5 = Node('ENTm5 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm6 = Node('ENTm6 inVoxel=False  n_exp= NA', parent=ENTm)
ENTmv = Node('ENTmv inVoxel=False  n_exp= NA', parent=ENT)
POST = Node('POST inVoxel=True  n_exp= NA', parent=RHP)
PRE = Node('PRE inVoxel=True  n_exp= NA', parent=RHP)
ProS = Node('ProS inVoxel=False  n_exp= NA', parent=RHP)
HATA = Node('HATA inVoxel=False  n_exp= NA', parent=RHP)
HIP = Node('HIP inVoxel=False  n_exp= NA', parent=HPF)
IG = Node('IG inVoxel=True  n_exp= NA', parent=HIP)
CA = Node('CA inVoxel=False  n_exp= NA', parent=HIP)
CA1 = Node('CA1 inVoxel=True  n_exp= NA', parent=CA)
CA2 = Node('CA2 inVoxel=True  n_exp= NA', parent=CA)
CA3 = Node('CA3 inVoxel=True  n_exp= NA', parent=CA)
DG = Node('DG inVoxel=True  n_exp= NA', parent=HIP)
DG_po = Node('DG_po inVoxel=False  n_exp= NA', parent=DG)
DG_sg = Node('DG_sg inVoxel=False  n_exp= NA', parent=DG)
DG_mo = Node('DG_mo inVoxel=False  n_exp= NA', parent=DG)
FC = Node('FC inVoxel=True  n_exp= NA', parent=HIP)
CTXsp = Node('CTXsp inVoxel=False  n_exp= NA', parent=CTX)
LA = Node('LA inVoxel=True  n_exp= NA', parent=CTXsp)
BLA = Node('BLA inVoxel=True  n_exp= NA', parent=CTXsp)
BLAa = Node('BLAa inVoxel=False  n_exp= NA', parent=BLA)
BLAp = Node('BLAp inVoxel=False  n_exp= NA', parent=BLA)
BLAv = Node('BLAv inVoxel=False  n_exp= NA', parent=BLA)
BMA = Node('BMA inVoxel=True  n_exp= NA', parent=CTXsp)
BMAa = Node('BMAa inVoxel=False  n_exp= NA', parent=BMA)
BMAp = Node('BMAp inVoxel=False  n_exp= NA', parent=BMA)
CLA = Node('CLA inVoxel=True  n_exp= NA', parent=CTXsp)
PA = Node('PA inVoxel=True  n_exp= NA', parent=CTXsp)
EP = Node('EP inVoxel=False  n_exp= NA', parent=CTXsp)
EPd = Node('EPd inVoxel=True  n_exp= NA', parent=EP)
EPv = Node('EPv inVoxel=True  n_exp= NA', parent=EP)

In [58]:
for pre, fill, node in RenderTree(CTX):
    print   pre,node.name

 CTX inVoxel=False
├──  CTXpl inVoxel=False  n_exp= NA
│   └──  Isocortex inVoxel=False  n_exp= NA
│       ├──  PTLp inVoxel=False  n_exp= NA
│       │   ├──  VISa inVoxel=True  n_exp= NA
│       │   │   ├──  VISa1 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa2and3 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa4 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa5 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa6a inVoxel=False  n_exp= NA
│       │   │   └──  VISa6b inVoxel=False  n_exp= NA
│       │   └──  VISrl inVoxel=True  n_exp= NA
│       │       ├──  VISrl1 inVoxel=False  n_exp= NA
│       │       ├──  VISrl2and3 inVoxel=False  n_exp= NA
│       │       ├──  VISrl4 inVoxel=False  n_exp= NA
│       │       ├──  VISrl5 inVoxel=False  n_exp= NA
│       │       ├──  VISrl6a inVoxel=False  n_exp= NA
│       │       └──  VISrl6b inVoxel=False  n_exp= NA
│       ├──  ACA inVoxel=False  n_exp= NA
│       │   ├──  ACAd inVoxel=True  n_exp= NA
│       │   │   ├──  ACAd2and3 inVo

From the tree, we know  ---

In [92]:
BS_nuclei_Damasio={'BS-d':['SCs',
                   'SCm',
                   'PB',
                   'NTS',
                   'PAG',
                   'AP']}

BS_nuclei={'BS':[id_to_acronym[i] for i in Unpack(acronym_to_id['BS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

TH_nuclei={'TH':[id_to_acronym[i] for i in Unpack(acronym_to_id['TH'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

HY_nuclei={'HY':[id_to_acronym[i] for i in Unpack(acronym_to_id['HY'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

CTX_nuclei={'CTX':[id_to_acronym[i] for i in Unpack(acronym_to_id['CTX'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

Iso_nuclei={'ISO':[id_to_acronym[i] for i in Unpack(acronym_to_id['Isocortex'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

ILM_nuclei={'ILM':[id_to_acronym[i] for i in Unpack(acronym_to_id['ILM'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}


#vision
VIS_nuclei={'VIS':[id_to_acronym[i] for i in Unpack(acronym_to_id['VIS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#hearing
AUD_nuclei={'AUD':[id_to_acronym[i] for i in Unpack(acronym_to_id['AUD'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#somatosensory (touch)
SS_nuclei={'SS':[id_to_acronym[i] for i in Unpack(acronym_to_id['SS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#taste
GU_nuclei={'GU':[id_to_acronym[i] for i in Unpack(acronym_to_id['GU'],Substr,include_self=True)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#smell
OLF_nuclei={'OLF':[id_to_acronym[i] for i in Unpack(acronym_to_id['OLF'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}


In [93]:
def Voxel_infoflow(src,tg,dens,avgfir,avgenc,VMacr_to_error,ddens=46000.0,davgfir=1.0,davgenc=0.5,return_mode='raw',reverse=False,print_Latex_Table=False):
    """Calculates the information flow between each element of the source and target lists.
    
    args:
    src (dict): dictionary of cardinality 1. maps the structure (super) name to its nuclei
    tg (dict): same as src for target
    dens (int): number of neurons per volume (neurons/mm^3)
    avgfir (float): neuron average firing rate per second (firing/s)
    avgenc (float): average number of bits encoded per action potential(bit)
    davgfir (float): error in avgfir
    davgenc (float): error in avgenc
    VMacr_to_error (dict): map from acronym of VM superstructures to the relative error. (for source)
    
    return_mode (str): choose return mode: 'raw','source nuclei to target','source to target'. Default='raw'
    print_Latex_Table (bool): print the formatted table usable for Latex if true. Default=False
    
    returns:
        dictionary containing projections from source to target. Dependant on return_mode
    """
    
    src_name=src.keys()[0] # fetch name of the source. Necessary since the original src is a dictionary
    tg_name=tg.keys()[0] # fetch name of the target
    src=src.values()[0] # list of source acronyms
    tg=tg.values()[0] #list of target acronyms
    

    proj_dict={}
    proj_error_dict={}
    for a in src:
        if a not in proj_dict:
            proj_dict[a]={}
            proj_error_dict[a]={}

        for b in tg:
            proj_value=Voxel_connectivity_dict[('ipsi',a)][b]*dens*avgfir*avgenc
            proj_dict[a][b]=proj_value

            proj_error_value=calc_MSE_error(a,VMacr_to_error,dens,avgfir,avgenc,ddens,davgfir,davgenc)
            proj_error_dict[a][b]=proj_value*proj_error_value
            
    # mode 1: raw
    if return_mode=='raw':
        if print_Latex_Table:
            printLatexTable(proj_dict,proj_error_dict)
        return proj_dict,proj_error_dict
    
    # mode 2" source nuclei to target
    elif return_mode=='source nuclei to target':
        proj_dict_mode2={}
        proj_error_dict_mode2={}
        for a in proj_dict:
            proj_dict_mode2[a]={}
            proj_error_dict_mode2[a]={}
            proj_value=0.0
            proj_error_value=0.0
            for b in proj_dict[a]:
                proj_value+=proj_dict[a][b]
                proj_error_value+=proj_error_dict[a][b]
            proj_dict_mode2[a][tg_name]=proj_value
            proj_error_dict_mode2[a][tg_name]=proj_error_value
        
        if print_Latex_Table:
            printLatexTable(proj_dict_mode2,proj_error_dict_mode2)
            
        return proj_dict_mode2,proj_error_dict_mode2  
    
    
    # return mode 4: source to target nuclei
    elif return_mode=='source to target nuclei':
        proj_dict_mode4={}
        proj_error_dict_mode4={}
        proj_dict_mode4[src_name]={}
        proj_error_dict_mode4[src_name]={}
        for b in tg:
            proj_dict_mode4[src_name][b]=0.0
            proj_error_dict_mode4[src_name][b]=0.0
        
        for a in tg:
            proj_value=0.0
            proj_error_value=0.0
            for b in src:
                proj_value+=proj_dict[b][a]
                proj_error_value+=proj_error_dict[b][a]
            proj_dict_mode4[src_name][a]=proj_value
            proj_error_dict_mode4[src_name][a]=proj_error_value
        
        if print_Latex_Table:
            printLatexTable(proj_dict_mode4,proj_error_dict_mode4)
            
        return proj_dict_mode4,proj_error_dict_mode4    
        
        
    # return mode 3: whole source to whole target
    elif return_mode=='source to target':
        proj_value=0.0
        proj_error_value=0.0
        for a in proj_dict:
            for b in proj_dict[a]:
                proj_value+=proj_dict[a][b]
                proj_error_value+=proj_error_dict[a][b]
        proj_dict_mode3={src_name:{tg_name:proj_value}}
        proj_error_dict_mode3={src_name:{tg_name:proj_error_value}}
                               
        if print_Latex_Table:
            printLatexTable(proj_dict_mode3,proj_error_dict_mode3)
        return proj_dict_mode3,proj_error_dict_mode3
    
    else:
        raise ValueError("Specify an acceptable return mode ('raw','source nuclei to target','source to target')")
    

In [94]:
def calc_MSE_error(src_acr,VMacr_to_error,dens,avgfir,avgenc,ddens,davgfir,davgenc):
    """
    args:
        src_acr (str): acronym of the source structure to calculate uncertainty of
        Voxel_superStructure_error(dict: list of superstructures with uncertainty given by the high resoluton paper
        davgfir (float): error in avgfir
        davgenc (float): error in davgenc
        """
    import math
    
    belong=which_do_I_belong(src_acr,VMacr_to_error)
    MSEerror=math.sqrt((VMacr_to_error[belong]/100)**2+(ddens/dens)**2+(davgfir/avgfir)**2+(davgenc/avgfir)**2)
    return MSEerror

In [95]:
def which_do_I_belong(src_acr,VMacr_to_error):
    """calculates whether a structure region is in the superstructure
    args:
        src_arc (str): acronym of the source structure
        superstructure (str): acronym of the super structure
    returns:
        acronym of the structure in which src_acr belongs to"""
    VMacr_superstructures=VMacr_to_error.keys()
    for i in VMacr_superstructures:
        if acronym_to_id[src_acr] in [j for j in Unpack(acronym_to_id[i],Substr,include_self=True)]:
            return i
        
    raise ValueError("No structure found. Check input")


In [96]:
def printLatexTable(proj_dict,proj_error_dict):
    import math
    
    proj_dict_keys=proj_dict.keys()[0]
    proj_error_dict_keys=proj_error_dict.keys()[0]
    
    if proj_dict.keys()!=proj_error_dict.keys():
        raise ValueError("keys of proj_dict and proj_error_dict not the same.")
    
    for a in proj_dict:
        for b in proj_dict[a]:
            proj_value=proj_dict[a][b]
            proj_error_value=proj_error_dict[a][b]
            print a+" & $\\rightarrow$ &"+b+" & "+str(int(proj_value))+" $\pm$ "+str(int(proj_error_value))+" & "+str(int(100*proj_error_value/proj_value))+" \\\\"

In [97]:
Voxel_Model_Table1_Superstructures=['Isocortex','Olfactory areas','Cortical subplate',
                                    'Hippocampal region','Striatum','Pallidum','Thalamus',
                                    'Hypothalamus','Midbrain','Pons','Medulla','Cerebellum']

Voxel_Model_Table1_RegionMSErel=[float(i) for i in [33,41,104,56,46,71,59,28,30,52,34,104]]
VMacr_to_error={}

for i in range(len(Voxel_Model_Table1_RegionMSErel)):
    VMacr_to_error[name_to_acronym[Voxel_Model_Table1_Superstructures[i]]]=Voxel_Model_Table1_RegionMSErel[i]

### SCs to sensory cortex

In [98]:
for i in ['VIS','AUD','SS','GU','OLF']:
    print "abc=Voxel_infoflow(BS_nuclei_Damasio,"+i+"_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)"


abc=Voxel_infoflow(BS_nuclei_Damasio,VIS_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,AUD_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,SS_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,GU_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,OLF_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)


In [99]:
abc=Voxel_infoflow(BS_nuclei_Damasio,VIS_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,AUD_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,SS_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,GU_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(BS_nuclei_Damasio,OLF_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)

SCm & $\rightarrow$ &VIS & 848522819 $\pm$ 685415283 & 80 \\
NTS & $\rightarrow$ &VIS & 75292 $\pm$ 62000 & 82 \\
PB & $\rightarrow$ &VIS & 1136381 $\pm$ 1037100 & 91 \\
AP & $\rightarrow$ &VIS & 59353 $\pm$ 48875 & 82 \\
PAG & $\rightarrow$ &VIS & 95389365 $\pm$ 77053118 & 80 \\
SCs & $\rightarrow$ &VIS & 676233018 $\pm$ 546243938 & 80 \\
SCm & $\rightarrow$ &AUD & 504836781 $\pm$ 407794390 & 80 \\
NTS & $\rightarrow$ &AUD & 55142 $\pm$ 45408 & 82 \\
PB & $\rightarrow$ &AUD & 2592223 $\pm$ 2365750 & 91 \\
AP & $\rightarrow$ &AUD & 6675 $\pm$ 5496 & 82 \\
PAG & $\rightarrow$ &AUD & 80512385 $\pm$ 65035870 & 80 \\
SCs & $\rightarrow$ &AUD & 62770327 $\pm$ 50704283 & 80 \\
SCm & $\rightarrow$ &SS & 1706525129 $\pm$ 1378487860 & 80 \\
NTS & $\rightarrow$ &SS & 17744807 $\pm$ 14612286 & 82 \\
PB & $\rightarrow$ &SS & 55129565 $\pm$ 50313091 & 91 \\
AP & $\rightarrow$ &SS & 179117 $\pm$ 147497 & 82 \\
PAG & $\rightarrow$ &SS & 377449619 $\pm$ 304894260 & 80 \\
SCs & $\rightarrow$ &SS & 4511

In [100]:
for i in ['VIS','AUD','SS','GU','OLF']:
    print "abc=Voxel_infoflow("+i+"_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)"

abc=Voxel_infoflow(VIS_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(AUD_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(SS_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(GU_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(OLF_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)


In [101]:
abc=Voxel_infoflow(VIS_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(AUD_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(SS_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(GU_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)
abc=Voxel_infoflow(OLF_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',reverse=False,print_Latex_Table=True)

VIS & $\rightarrow$ &SCm & 5589322 $\pm$ 4579834 & 81 \\
VIS & $\rightarrow$ &NTS & 515463 $\pm$ 422365 & 81 \\
VIS & $\rightarrow$ &PB & 233940 $\pm$ 191688 & 81 \\
VIS & $\rightarrow$ &AP & 110967 $\pm$ 90925 & 81 \\
VIS & $\rightarrow$ &PAG & 653769 $\pm$ 535692 & 81 \\
VIS & $\rightarrow$ &SCs & 3372878 $\pm$ 2763703 & 81 \\
AUD & $\rightarrow$ &SCm & 4137475 $\pm$ 3390205 & 81 \\
AUD & $\rightarrow$ &NTS & 442532 $\pm$ 362606 & 81 \\
AUD & $\rightarrow$ &PB & 96376 $\pm$ 78969 & 81 \\
AUD & $\rightarrow$ &AP & 93748 $\pm$ 76816 & 81 \\
AUD & $\rightarrow$ &PAG & 470174 $\pm$ 385255 & 81 \\
AUD & $\rightarrow$ &SCs & 429500 $\pm$ 351928 & 81 \\
SS & $\rightarrow$ &SCm & 6455230 $\pm$ 5289351 & 81 \\
SS & $\rightarrow$ &NTS & 2835207 $\pm$ 2323140 & 81 \\
SS & $\rightarrow$ &PB & 1054527 $\pm$ 864069 & 81 \\
SS & $\rightarrow$ &AP & 505044 $\pm$ 413828 & 81 \\
SS & $\rightarrow$ &PAG & 2952412 $\pm$ 2419176 & 81 \\
SS & $\rightarrow$ &SCs & 2387722 $\pm$ 1956476 & 81 \\
GU & $\right

### thalamic nuclei to sensory

In [104]:
DORsm_nuclei={'DORsm':[id_to_acronym[i] for i in Unpack(acronym_to_id['DORsm'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
DORpm_nuclei={'DORpm':[id_to_acronym[i] for i in Unpack(acronym_to_id['DORpm'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
ILM_nuclei={'ILM':[id_to_acronym[i] for i in Unpack(acronym_to_id['ILM'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

In [111]:
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(i,DORsm_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(DORsm_nuclei,i,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)

VIS & $\rightarrow$ &DORsm & 1275368237 $\pm$ 1045024001 & 81 \\
AUD & $\rightarrow$ &DORsm & 515198281 $\pm$ 422148328 & 81 \\
SS & $\rightarrow$ &DORsm & 2980057353 $\pm$ 2441829245 & 81 \\
GU & $\rightarrow$ &DORsm & 60475946 $\pm$ 49553386 & 81 \\
OLF & $\rightarrow$ &DORsm & 35201260 $\pm$ 30088327 & 85 \\
DORsm & $\rightarrow$ &VIS & 848237266 $\pm$ 809433495 & 95 \\
DORsm & $\rightarrow$ &AUD & 617794831 $\pm$ 589532963 & 95 \\
DORsm & $\rightarrow$ &SS & 3828626743 $\pm$ 3653480987 & 95 \\
DORsm & $\rightarrow$ &GU & 131179099 $\pm$ 125178132 & 95 \\
DORsm & $\rightarrow$ &OLF & 26344804 $\pm$ 25139625 & 95 \\


In [112]:
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(i,DORpm_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(DORpm_nuclei,i,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)

VIS & $\rightarrow$ &DORpm & 1269178486 $\pm$ 1039952182 & 81 \\
AUD & $\rightarrow$ &DORpm & 427360914 $\pm$ 350175267 & 81 \\
SS & $\rightarrow$ &DORpm & 2374099618 $\pm$ 1945313526 & 81 \\
GU & $\rightarrow$ &DORpm & 52444378 $\pm$ 42972400 & 81 \\
OLF & $\rightarrow$ &DORpm & 227571147 $\pm$ 194516762 & 85 \\
DORpm & $\rightarrow$ &VIS & 2699871059 $\pm$ 2576361773 & 95 \\
DORpm & $\rightarrow$ &AUD & 500759762 $\pm$ 477851823 & 95 \\
DORpm & $\rightarrow$ &SS & 3340834255 $\pm$ 3188003232 & 95 \\
DORpm & $\rightarrow$ &GU & 134321195 $\pm$ 128176488 & 95 \\
DORpm & $\rightarrow$ &OLF & 229106073 $\pm$ 218625303 & 95 \\


In [114]:
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(ILM_nuclei,i,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    abc=Voxel_infoflow(i,ILM_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',reverse=False,print_Latex_Table=True)


ILM & $\rightarrow$ &VIS & 51513808 $\pm$ 49157239 & 95 \\
ILM & $\rightarrow$ &AUD & 8675439 $\pm$ 8278569 & 95 \\
ILM & $\rightarrow$ &SS & 532841065 $\pm$ 508465523 & 95 \\
ILM & $\rightarrow$ &GU & 32445678 $\pm$ 30961406 & 95 \\
ILM & $\rightarrow$ &OLF & 5030500 $\pm$ 4800373 & 95 \\
VIS & $\rightarrow$ &ILM & 14341683 $\pm$ 11751431 & 81 \\
AUD & $\rightarrow$ &ILM & 1732739 $\pm$ 1419789 & 81 \\
SS & $\rightarrow$ &ILM & 150536702 $\pm$ 123348271 & 81 \\
GU & $\rightarrow$ &ILM & 11220480 $\pm$ 9193949 & 81 \\
OLF & $\rightarrow$ &ILM & 16487572 $\pm$ 14092775 & 85 \\


In [116]:
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    print acronym_to_name[i.keys()[0]]

Visual areas
Auditory areas
Somatosensory areas
Gustatory areas
Olfactory areas


In [118]:
acronym_to_name['DORpm']

u'Thalamus, polymodal association cortex related'

### BS Damasio to TH&HY

In [449]:
abc=Voxel_infoflow(BS_nuclei_Damasio,TH_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',reverse=False,print_Latex_Table=True)

SCm & $\rightarrow$ &TH & 46537047 $\pm$ 37591450 & 80 \\
NTS & $\rightarrow$ &TH & 59309 $\pm$ 48839 & 82 \\
PB & $\rightarrow$ &TH & 2283944 $\pm$ 2084404 & 91 \\
AP & $\rightarrow$ &TH & 24920 $\pm$ 20521 & 82 \\
PAG & $\rightarrow$ &TH & 69136308 $\pm$ 55846562 & 80 \\
SCs & $\rightarrow$ &TH & 13520503 $\pm$ 10921521 & 80 \\


In [451]:
abc=Voxel_infoflow(BS_nuclei_Damasio,HY_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',print_Latex_Table=True)

SCm & $\rightarrow$ &HY & 353734972 $\pm$ 285738168 & 80 \\
NTS & $\rightarrow$ &HY & 4074540 $\pm$ 3355254 & 82 \\
PB & $\rightarrow$ &HY & 22172104 $\pm$ 20235006 & 91 \\
AP & $\rightarrow$ &HY & 119922 $\pm$ 98752 & 82 \\
PAG & $\rightarrow$ &HY & 552988870 $\pm$ 446690430 & 80 \\
SCs & $\rightarrow$ &HY & 10814385 $\pm$ 8735587 & 80 \\


### TH&HY to BS Damasio

In [83]:
abc=Voxel_infoflow(TH_nuclei,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target nuclei',print_Latex_Table=True)

TH & $\rightarrow$ &SCm & 657884103 $\pm$ 627788296 & 95 \\
TH & $\rightarrow$ &NTS & 7050328 $\pm$ 6727801 & 95 \\
TH & $\rightarrow$ &PB & 45883522 $\pm$ 43784518 & 95 \\
TH & $\rightarrow$ &AP & 1358673 $\pm$ 1296519 & 95 \\
TH & $\rightarrow$ &PAG & 276555430 $\pm$ 263904025 & 95 \\
TH & $\rightarrow$ &SCs & 289874685 $\pm$ 276613972 & 95 \\


In [452]:
for i in BS_nuclei_Damasio[BS_nuclei_Damasio.keys()[0]]:
    abc=Voxel_infoflow(TH_nuclei,{str(i):[i]},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)

TH & $\rightarrow$ &SCs & 289874685 $\pm$ 276613972 & 95 \\
TH & $\rightarrow$ &SCm & 657884103 $\pm$ 627788296 & 95 \\
TH & $\rightarrow$ &PB & 45883522 $\pm$ 43784518 & 95 \\
TH & $\rightarrow$ &NTS & 7050328 $\pm$ 6727801 & 95 \\
TH & $\rightarrow$ &PAG & 276555430 $\pm$ 263904025 & 95 \\
TH & $\rightarrow$ &AP & 1358673 $\pm$ 1296519 & 95 \\


In [453]:
for i in BS_nuclei_Damasio[BS_nuclei_Damasio.keys()[0]]:
    abc=Voxel_infoflow(HY_nuclei,{str(i):[i]},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)

HY & $\rightarrow$ &SCs & 31010500 $\pm$ 24825837 & 80 \\
HY & $\rightarrow$ &SCm & 198470941 $\pm$ 158888353 & 80 \\
HY & $\rightarrow$ &PB & 71534580 $\pm$ 57267888 & 80 \\
HY & $\rightarrow$ &NTS & 6383071 $\pm$ 5110046 & 80 \\
HY & $\rightarrow$ &PAG & 238186492 $\pm$ 190683126 & 80 \\
HY & $\rightarrow$ &AP & 1302283 $\pm$ 1042559 & 80 \\


### Between BS Damasio

In [454]:
abc=Voxel_infoflow(BS_nuclei_Damasio,BS_nuclei_Damasio,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='raw',print_Latex_Table=True)

SCm & $\rightarrow$ &SCm & 1211803601 $\pm$ 978864316 & 80 \\
SCm & $\rightarrow$ &NTS & 1959667 $\pm$ 1582969 & 80 \\
SCm & $\rightarrow$ &PB & 16971372 $\pm$ 13709045 & 80 \\
SCm & $\rightarrow$ &AP & 261002 $\pm$ 210831 & 80 \\
SCm & $\rightarrow$ &PAG & 437980940 $\pm$ 353789931 & 80 \\
SCm & $\rightarrow$ &SCs & 512887383 $\pm$ 414297462 & 80 \\
NTS & $\rightarrow$ &SCm & 960614 $\pm$ 791035 & 82 \\
NTS & $\rightarrow$ &NTS & 46593371 $\pm$ 38368163 & 82 \\
NTS & $\rightarrow$ &PB & 3482119 $\pm$ 2867415 & 82 \\
NTS & $\rightarrow$ &AP & 6619582 $\pm$ 5451016 & 82 \\
NTS & $\rightarrow$ &PAG & 1273321 $\pm$ 1048539 & 82 \\
NTS & $\rightarrow$ &SCs & 28764 $\pm$ 23686 & 82 \\
PB & $\rightarrow$ &SCm & 22927734 $\pm$ 20924620 & 91 \\
PB & $\rightarrow$ &NTS & 17234980 $\pm$ 15729221 & 91 \\
PB & $\rightarrow$ &PB & 72582877 $\pm$ 66241569 & 91 \\
PB & $\rightarrow$ &AP & 1101810 $\pm$ 1005549 & 91 \\
PB & $\rightarrow$ &PAG & 24903375 $\pm$ 22727656 & 91 \\
PB & $\rightarrow$ &SCs &

### ISO to BS Damasio

In [455]:
for i in BS_nuclei_Damasio[BS_nuclei_Damasio.keys()[0]]:
    abc=Voxel_infoflow(Iso_nuclei,{str(i):[i]},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)

ISO & $\rightarrow$ &SCs & 32265290 $\pm$ 26437856 & 81 \\
ISO & $\rightarrow$ &SCm & 49158839 $\pm$ 40280261 & 81 \\
ISO & $\rightarrow$ &PB & 6540277 $\pm$ 5359037 & 81 \\
ISO & $\rightarrow$ &NTS & 8012953 $\pm$ 6565734 & 81 \\
ISO & $\rightarrow$ &PAG & 16523224 $\pm$ 13538965 & 81 \\
ISO & $\rightarrow$ &AP & 1487671 $\pm$ 1218982 & 81 \\


### BS Damasio to ISO

In [456]:
abc=Voxel_infoflow(BS_nuclei_Damasio,Iso_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',print_Latex_Table=True)

SCm & $\rightarrow$ &ISO & 7597631593 $\pm$ 6137174741 & 80 \\
NTS & $\rightarrow$ &ISO & 61908988 $\pm$ 50980087 & 82 \\
PB & $\rightarrow$ &ISO & 230872818 $\pm$ 210702280 & 91 \\
AP & $\rightarrow$ &ISO & 656016 $\pm$ 540209 & 82 \\
PAG & $\rightarrow$ &ISO & 3064475449 $\pm$ 2475405801 & 80 \\
SCs & $\rightarrow$ &ISO & 1254489523 $\pm$ 1013344924 & 80 \\


### ISO nuclei to TH + reverse

In [463]:
abc=Voxel_infoflow(Iso_nuclei,TH_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',print_Latex_Table=True)

VISrl & $\rightarrow$ &TH & 138176869 $\pm$ 113220747 & 81 \\
ILA & $\rightarrow$ &TH & 109567073 $\pm$ 89778166 & 81 \\
TEa & $\rightarrow$ &TH & 442547609 $\pm$ 362619093 & 81 \\
AUDpo & $\rightarrow$ &TH & 81012788 $\pm$ 66381069 & 81 \\
VISam & $\rightarrow$ &TH & 91843555 $\pm$ 75255692 & 81 \\
VISC & $\rightarrow$ &TH & 253922749 $\pm$ 208061766 & 81 \\
SSp-tr & $\rightarrow$ &TH & 171911032 $\pm$ 140862183 & 81 \\
VISpm & $\rightarrow$ &TH & 86844472 $\pm$ 71159494 & 81 \\
VISpl & $\rightarrow$ &TH & 27058282 $\pm$ 22171286 & 81 \\
AUDv & $\rightarrow$ &TH & 412115570 $\pm$ 337683384 & 81 \\
VISp & $\rightarrow$ &TH & 1733143387 $\pm$ 1420120390 & 81 \\
ACAd & $\rightarrow$ &TH & 418218352 $\pm$ 342683942 & 81 \\
AUDp & $\rightarrow$ &TH & 372909518 $\pm$ 305558336 & 81 \\
PERI & $\rightarrow$ &TH & 18449817 $\pm$ 15117596 & 81 \\
AIv & $\rightarrow$ &TH & 135183844 $\pm$ 110768292 & 81 \\
AIp & $\rightarrow$ &TH & 50196239 $\pm$ 41130297 & 81 \\
ACAv & $\rightarrow$ &TH & 31860

In [464]:
for i in Iso_nuclei[Iso_nuclei.keys()[0]]:
    abc=Voxel_infoflow(TH_nuclei,{str(i):[i]},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)

TH & $\rightarrow$ &VISa & 625054174 $\pm$ 596460218 & 95 \\
TH & $\rightarrow$ &VISrl & 225912073 $\pm$ 215577417 & 95 \\
TH & $\rightarrow$ &ACAd & 1316350306 $\pm$ 1256132065 & 95 \\
TH & $\rightarrow$ &ACAv & 950603658 $\pm$ 907116996 & 95 \\
TH & $\rightarrow$ &ILA & 391327418 $\pm$ 373425611 & 95 \\
TH & $\rightarrow$ &AId & 497991360 $\pm$ 475210065 & 95 \\
TH & $\rightarrow$ &AIp & 268931881 $\pm$ 256629225 & 95 \\
TH & $\rightarrow$ &AIv & 217821653 $\pm$ 207857104 & 95 \\
TH & $\rightarrow$ &FRP & 159076598 $\pm$ 151799422 & 95 \\
TH & $\rightarrow$ &AUDp & 433911041 $\pm$ 414061188 & 95 \\
TH & $\rightarrow$ &AUDd & 199856318 $\pm$ 190713618 & 95 \\
TH & $\rightarrow$ &AUDv & 345359857 $\pm$ 329560899 & 95 \\
TH & $\rightarrow$ &AUDpo & 139427376 $\pm$ 133049080 & 95 \\
TH & $\rightarrow$ &RSPd & 1336657646 $\pm$ 1275510417 & 95 \\
TH & $\rightarrow$ &RSPv & 1723978730 $\pm$ 1645112970 & 95 \\
TH & $\rightarrow$ &RSPagl & 858020457 $\pm$ 818769140 & 95 \\
TH & $\rightarrow$ 

### ILM

In [461]:
abc=Voxel_infoflow(BS_nuclei_Damasio,ILM_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source nuclei to target',print_Latex_Table=True)

SCm & $\rightarrow$ &ILM & 1596394 $\pm$ 1289527 & 80 \\
NTS & $\rightarrow$ &ILM & 3632 $\pm$ 2991 & 82 \\
PB & $\rightarrow$ &ILM & 114316 $\pm$ 104328 & 91 \\
AP & $\rightarrow$ &ILM & 1921 $\pm$ 1582 & 82 \\
PAG & $\rightarrow$ &ILM & 3597984 $\pm$ 2906361 & 80 \\
SCs & $\rightarrow$ &ILM & 354528 $\pm$ 286378 & 80 \\


In [462]:
for i in BS_nuclei_Damasio[BS_nuclei_Damasio.keys()[0]]:
    abc=Voxel_infoflow(ILM_nuclei,{str(i):[i]},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)

ILM & $\rightarrow$ &SCs & 13208135 $\pm$ 12603911 & 95 \\
ILM & $\rightarrow$ &SCm & 70425588 $\pm$ 67203873 & 95 \\
ILM & $\rightarrow$ &PB & 10019128 $\pm$ 9560789 & 95 \\
ILM & $\rightarrow$ &NTS & 792101 $\pm$ 755865 & 95 \\
ILM & $\rightarrow$ &PAG & 39911185 $\pm$ 38085394 & 95 \\
ILM & $\rightarrow$ &AP & 18391 $\pm$ 17550 & 95 \\


### Thalamus nuclei to ISO

In [466]:
%pprint
THsub=['LAT','ATN','MED','ILM','GENv','EPI']
for i in THsub:
    print i+" = "+acronym_to_name[i]

Pretty printing has been turned OFF
LAT = Lateral group of the dorsal thalamus
ATN = Anterior group of the dorsal thalamus
MED = Medial group of the dorsal thalamus
ILM = Intralaminar nuclei of the dorsal thalamus
GENv = Geniculate group, ventral thalamus
EPI = Epithalamus


In [457]:
for k in THsub:
    nuclei={k : [id_to_acronym[i] for i in Unpack(acronym_to_id[k],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
    Voxel_infoflow(nuclei,Iso_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)
Voxel_infoflow({'RT':['RT']},Iso_nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)
None

LAT & $\rightarrow$ &ISO & 6008313709 $\pm$ 5733455197 & 95 \\
ATN & $\rightarrow$ &ISO & 4378612551 $\pm$ 4178306943 & 95 \\
MED & $\rightarrow$ &ISO & 2655623539 $\pm$ 2534138415 & 95 \\
ILM & $\rightarrow$ &ISO & 2388278624 $\pm$ 2279023559 & 95 \\
GENv & $\rightarrow$ &ISO & 661003530 $\pm$ 630765021 & 95 \\
EPI & $\rightarrow$ &ISO & 201117644 $\pm$ 191917242 & 95 \\
RT & $\rightarrow$ &ISO & 2590144243 $\pm$ 2471654559 & 95 \\


In [460]:
for k in THsub:
    nuclei={k : [id_to_acronym[i] for i in Unpack(acronym_to_id[k],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
    
    abc=Voxel_infoflow(Iso_nuclei,nuclei,92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)
abc=Voxel_infoflow(Iso_nuclei,{'RT':['RT']},92000.0,2.0,1.0,VMacr_to_error,davgfir=1.0,davgenc=0.5,return_mode='source to target',print_Latex_Table=True)
None

ISO & $\rightarrow$ &LAT & 3048902266 $\pm$ 2498240079 & 81 \\
ISO & $\rightarrow$ &ATN & 1874500582 $\pm$ 1535947064 & 81 \\
ISO & $\rightarrow$ &MED & 1236798484 $\pm$ 1013420331 & 81 \\
ISO & $\rightarrow$ &ILM & 698809581 $\pm$ 572597595 & 81 \\
ISO & $\rightarrow$ &GENv & 674999876 $\pm$ 553088160 & 81 \\
ISO & $\rightarrow$ &EPI & 104793679 $\pm$ 85866894 & 81 \\
ISO & $\rightarrow$ &RT & 1242653697 $\pm$ 1018218034 & 81 \\


Create a list that contains Damasio's Network

In [41]:
from collections import Iterable

def flatten(lis):
    """Iterate over an arbitrary dimensional iterable and returns a list containing
    all the elements in the list"""
    
    for item in lis:
        if isinstance(item,Iterable) and not isinstance(item,basestring):
            for x in flatten(item):
                yield x
        else:
            yield item

    

In [42]:
Damasio_Network=[['SCs','SCm'],
                ['PB'],
                ['NTS'],
                ['PAG'],
                ['AP'],
                ]

# add nuclei of the hypothalamus
for i in Substr[acronym_to_id['HY']]:
    toappend=[str(id_to_acronym[j]) for j in Substr[i]]
    Damasio_Network.append(toappend)

#### From HY to BS

In [43]:
flat_Damasio=list(flatten(Damasio_Network))

In [44]:
flat_Damasio

['SCs', 'SCm', 'PB', 'NTS', 'PAG', 'AP', 'ADP', 'PVp', 'PVpo', 'VLPO', 'AVP', 'AVPV', 'SCH', 'SFO', 'SBPV', 'MEPO', 'MPO', 'OV', 'DMH', 'PD', 'PS', 'VMPO', 'PVa', 'PVH', 'PVi', 'ARH', 'ASO', 'SO', 'RCH', 'LHA', 'LPO', 'PST', 'PSTN', 'STN', 'TU', 'ZI', 'PeF', 'PVHd', 'AHN', 'MBO', 'MPN', 'VMH', 'PH', 'PMd', 'PMv']

In [46]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[i]=0
            else:
                exception_dict[i]+=1

In [47]:
exception_dict

{'PeF': 5, 'MBO': 5, 'VMPO': 5}

In [48]:
HY_to_BS={}
for i in flat_Damasio[:6]:
    HY_to_BS[('HY',i)]=0

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        if i not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            HY_to_BS[('HY',j)]+=add

In [49]:
exception_dict.keys()

['PeF', 'MBO', 'VMPO']

In [51]:
%pprint

Pretty printing has been turned ON


In [73]:
for i in HY_to_BS:
    print i[0]+" & $\\rightarrow$ &"+i[1]+" & "+str(round(HY_to_BS[i],1))+" \\\\"

HY & $\rightarrow$ &SCm & 98010599.8 \\
HY & $\rightarrow$ &PAG & 112840524.6 \\
HY & $\rightarrow$ &NTS & 2888747.2 \\
HY & $\rightarrow$ &SCs & 15164716.9 \\
HY & $\rightarrow$ &PB & 35070641.3 \\
HY & $\rightarrow$ &AP & 601992.8 \\


#### From BS to HY

In [53]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[j]=0
            else:
                exception_dict[j]+=1

In [54]:
Damasio_dict

{('AP', 'ADP'): 312.9441143682925,
 ('AP', 'AHN'): 5135.985342581989,
 ('AP', 'ARH'): 3020.6431963597424,
 ('AP', 'ASO'): 17.178109206724912,
 ('AP', 'AVP'): 173.4274537739111,
 ('AP', 'AVPV'): 355.974893682287,
 ('AP', 'DMH'): 3795.1260693953373,
 ('AP', 'LHA'): 11433.845429819485,
 ('AP', 'LPO'): 1416.3174082677872,
 ('AP', 'MEPO'): 205.06868507163747,
 ('AP', 'MPN'): 2100.520661333576,
 ('AP', 'MPO'): 1860.2410752355354,
 ('AP', 'OV'): 22.933276151889004,
 ('AP', 'PD'): 57.28061546687968,
 ('AP', 'PH'): 2095.3289983444847,
 ('AP', 'PMd'): 587.8298965108115,
 ('AP', 'PMv'): 1563.8487985997926,
 ('AP', 'PS'): 310.3679866471793,
 ('AP', 'PST'): 45.41565314866603,
 ('AP', 'PSTN'): 336.65115421536024,
 ('AP', 'PVH'): 1833.1710154307077,
 ('AP', 'PVHd'): 1210.4691684362479,
 ('AP', 'PVa'): 19.303322042105716,
 ('AP', 'PVi'): 588.5014634695835,
 ('AP', 'PVp'): 790.4466252075508,
 ('AP', 'PVpo'): 546.6821859154152,
 ('AP', 'RCH'): 876.9786358170678,
 ('AP', 'SBPV'): 1063.4426152973904,
 ('A

In [55]:
exception_dict

{'MBO': 0, 'PeF': 0, 'VMPO': 0}

In [56]:
BS_to_HY={}
for i in flat_Damasio[:6]:
    BS_to_HY[('BS',i)]=0

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        if j not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            BS_to_HY[('BS',i)]+=add

In [57]:
BS_to_HY

{('BS', 'AP'): 57174.72434903448,
 ('BS', 'NTS'): 1940701.527778525,
 ('BS', 'PAG'): 260257660.69746017,
 ('BS', 'PB'): 10665332.4003499,
 ('BS', 'SCm'): 173464864.34695125,
 ('BS', 'SCs'): 5235818.558663945}

In [58]:
HY_to_BS

{('HY', 'AP'): 601992.8089413116,
 ('HY', 'NTS'): 2888747.214828966,
 ('HY', 'PAG'): 112840524.61648126,
 ('HY', 'PB'): 35070641.28208359,
 ('HY', 'SCm'): 98010599.76085003,
 ('HY', 'SCs'): 15164716.876646407}

#### error

In [100]:
import math

In [101]:
dfiring=1/2
dencode=0.5/1
dreg={'MED':0.34,'PONS':0.52,'MID':0.30,'TH':0.59,'HY':0.28}

belong={'SCs':'MID',
        'SCm':'MID',
        'PB':'PONS',
        'NTS':'MED',
        'PAG':'MID',
        'AP':'MED',
        }

In [165]:
for i in HY_to_BS:
    error=math.sqrt(0.28**2+dfiring**2+dencode**2)
    value=2*int(HY_to_BS[i])
    print "HY"+" & $\\rightarrow$ &"+i[1]+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

HY & $\rightarrow$ &SCm & 196021198 $\pm$ 112332290 & 57 \\
HY & $\rightarrow$ &PAG & 225681048 $\pm$ 129329221 & 57 \\
HY & $\rightarrow$ &NTS & 5777494 $\pm$ 3310861 & 57 \\
HY & $\rightarrow$ &SCs & 30329432 $\pm$ 17380643 & 57 \\
HY & $\rightarrow$ &PB & 70141282 $\pm$ 40195299 & 57 \\
HY & $\rightarrow$ &AP & 1203984 $\pm$ 689957 & 57 \\


In [166]:
for i in BS_to_HY:
    error=math.sqrt(dreg[belong[i[1]]]**2+dfiring**2+dencode**2)
    value=2*int(BS_to_HY[i])
    print i[1]+" & $\\rightarrow$ &"+'HY'+" & "+str(value)+"$\pm$"+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

SCs & $\rightarrow$ &HY & 10471636$\pm$6105960 & 58 \\
PB & $\rightarrow$ &HY & 21330664$\pm$15387675 & 72 \\
NTS & $\rightarrow$ &HY & 3881402$\pm$2346884 & 60 \\
AP & $\rightarrow$ &HY & 114348$\pm$69140 & 60 \\
PAG & $\rightarrow$ &HY & 520515320$\pm$303509979 & 58 \\
SCm & $\rightarrow$ &HY & 346929728$\pm$202293055 & 58 \\


## Within BS

In [62]:
WithinBS=flat_Damasio[:6]
BS_dict={}

In [75]:
for i in WithinBS:
    for j in WithinBS:
        proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
        if i!=j:
            BS_dict[(i,j)]=proj

In [77]:
BS_dict

{('AP', 'NTS'): 712244.0012404694,
 ('AP', 'PAG'): 15497.182208262187,
 ('AP', 'PB'): 27687.67467410362,
 ('AP', 'SCm'): 3533.0245365854016,
 ('AP', 'SCs'): 631.0906367403195,
 ('NTS', 'AP'): 3309791.162014007,
 ('NTS', 'PAG'): 636660.5903630842,
 ('NTS', 'PB'): 1741059.8600879314,
 ('NTS', 'SCm'): 480307.418823344,
 ('NTS', 'SCs'): 14382.003381243065,
 ('PAG', 'AP'): 218175.6619513035,
 ('PAG', 'NTS'): 2120243.2674914603,
 ('PAG', 'PB'): 27486981.784284115,
 ('PAG', 'SCm'): 294961847.85437584,
 ('PAG', 'SCs'): 61730128.873944275,
 ('PB', 'AP'): 550905.2445292473,
 ('PB', 'NTS'): 8617490.158677101,
 ('PB', 'PAG'): 12451687.776265666,
 ('PB', 'SCm'): 11463867.496000603,
 ('PB', 'SCs'): 856972.5450677796,
 ('SCm', 'AP'): 130501.18985027073,
 ('SCm', 'NTS'): 979833.6065299809,
 ('SCm', 'PAG'): 218990470.1816738,
 ('SCm', 'PB'): 8485686.30076945,
 ('SCm', 'SCs'): 256443691.5473938,
 ('SCs', 'AP'): 54578.63790169358,
 ('SCs', 'NTS'): 247307.53509607166,
 ('SCs', 'PAG'): 27593840.889549814,


In [ ]:
for i in HY_to_BS:
    error=float(0.28**2+dfiring**2+dencode**2)
    value=2*int(HY_to_BS[i])
    print i+" & $\\rightarrow$ &"+j+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

In [122]:
tlist=['AP','NTS','PAG','PB','SCm','SCs']
for i in tlist:
    for j in tlist:
        if i!=j:
            error=math.sqrt(dreg[belong[i]]**2+dfiring**2+dencode**2)
            value=2*int(BS_dict[(i,j)])
            print i+" & $\\rightarrow$ &"+j+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+   " \\\\"

AP & $\rightarrow$ &NTS & 1424488 $\pm$ 861314 & 60 \\
AP & $\rightarrow$ &PAG & 30994 $\pm$ 18740 & 60 \\
AP & $\rightarrow$ &PB & 55374 $\pm$ 33481 & 60 \\
AP & $\rightarrow$ &SCm & 7066 $\pm$ 4272 & 60 \\
AP & $\rightarrow$ &SCs & 1262 $\pm$ 763 & 60 \\
NTS & $\rightarrow$ &AP & 6619582 $\pm$ 4002521 & 60 \\
NTS & $\rightarrow$ &PAG & 1273320 $\pm$ 769911 & 60 \\
NTS & $\rightarrow$ &PB & 3482118 $\pm$ 2105457 & 60 \\
NTS & $\rightarrow$ &SCm & 960614 $\pm$ 580833 & 60 \\
NTS & $\rightarrow$ &SCs & 28764 $\pm$ 17392 & 60 \\
PAG & $\rightarrow$ &AP & 436350 $\pm$ 254433 & 58 \\
PAG & $\rightarrow$ &NTS & 4240486 $\pm$ 2472606 & 58 \\
PAG & $\rightarrow$ &PB & 54973962 $\pm$ 32055052 & 58 \\
PAG & $\rightarrow$ &SCm & 589923694 $\pm$ 343981668 & 58 \\
PAG & $\rightarrow$ &SCs & 123460256 $\pm$ 71989081 & 58 \\
PB & $\rightarrow$ &AP & 1101810 $\pm$ 794832 & 72 \\
PB & $\rightarrow$ &NTS & 17234980 $\pm$ 12433100 & 72 \\
PB & $\rightarrow$ &PAG & 24903374 $\pm$ 17964984 & 72 \\
PB & $\

## Between CTX and HypoBS

We want to be able to create a table of information flow in terms of 'large' cortical regions, such as VIS, AUD etc. So we need a way to manage that. 

The previously defined Substr and Unpack may come in handy here



In [131]:
Voxel_structures.shape

(293, 8)

In [180]:
def Cortical_HYBS_map(ctx_region,target):
    """Calculates the Projection sum from the specific cortical regions into the BS or the HyThalamus
    args:
        ctx_region(basestring): id of the structure
        target(basestring): id of the target structure"""
    
    source=[id_to_acronym[i] for i in Unpack(acronym_to_id[ctx_region],Substr,include_self=False)] 
            #list of id
    
    #if it has no substructure
    #if len(source)==0:
    #    source=Unpack(ctx_region,Substr,include_self=True)
    
    Voxel_source=[]
    for i in source:
        if i in list(Voxel_structures['acronym']):
            Voxel_source.append(i)
    
    ret=0
    for i in Voxel_source:
        ret+=Voxel_connectivity_dict[('ipsi',i)][target]*92000
    return ret
    

In [181]:
CTX_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp']
BS_nuclei=['SCs', 'SCm','PB','NTS','PAG','AP']

CTX_to_BS_nuclei={}
for i in BS_nuclei:
    CTX_to_BS_nuclei[i]=0
    
sum_me=0
for i in BS_nuclei:
    for j in CTX_regions:
        addme=Cortical_HYBS_map(j,i)
        sum_me+=addme
    CTX_to_BS_nuclei[i]=sum_me
    sum_me=0

In [135]:
CTX_to_BS_nuclei

{'AP': 565280.6168194802,
 'NTS': 3127679.3021562,
 'PAG': 7156091.618768791,
 'PB': 2035798.1088416118,
 'SCm': 23363330.75415419,
 'SCs': 15872781.214532183}

In [169]:
for i in CTX_to_BS_nuclei:
    error=math.sqrt(0.33**2+dfiring**2+dencode**2)
    value=2*int(CTX_to_BS_nuclei[i])
    print "ISO"+" & $\\rightarrow$ &"+i+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

ISO & $\rightarrow$ &SCm & 46726660 $\pm$ 27993130 & 59 \\
ISO & $\rightarrow$ &NTS & 6255358 $\pm$ 3747476 & 59 \\
ISO & $\rightarrow$ &PB & 4071596 $\pm$ 2439222 & 59 \\
ISO & $\rightarrow$ &AP & 1130560 $\pm$ 677298 & 59 \\
ISO & $\rightarrow$ &PAG & 14312182 $\pm$ 8574179 & 59 \\
ISO & $\rightarrow$ &SCs & 31745562 $\pm$ 19018214 & 59 \\


In [177]:
Unpack(acronym_to_id['TH'],Substr)

[856,
 51,
 189,
 575,
 599,
 907,
 930,
 560581563,
 138,
 218,
 325,
 1020,
 1029,
 560581551,
 239,
 64,
 127,
 1096,
 1104,
 155,
 255,
 1113,
 1120,
 262,
 444,
 59,
 362,
 366,
 1077,
 571,
 15,
 149,
 181,
 560581559,
 958,
 186,
 483,
 1014,
 27,
 178,
 321,
 563807439,
 864,
 406,
 414,
 422,
 609,
 637,
 685,
 563807435,
 629,
 709,
 718,
 725,
 733,
 741,
 1008,
 170,
 496345664,
 496345668,
 496345672,
 475,
 1072,
 1079,
 1088,
 1044]

CTX into HYpothalamus

In [344]:
sum(CTX_to_HY_nuclei.values())

129265943.04776353

In [182]:
CTX_regions=['AUD','VIS','MO','SS','ACA','ORB','AI','RSP','PTLp']
HY_nuclei=flat_Damasio[6:]

exception_list=['VMPO','PeF','MBO']


CTX_to_HY_nuclei={}
for i in HY_nuclei:
    CTX_to_HY_nuclei[i]=0
    
sum_me=0
for i in HY_nuclei:
    for j in CTX_regions:
        if i not in exception_list:
            addme=Cortical_HYBS_map(j,i)
            sum_me+=addme
        else:
            None
    CTX_to_HY_nuclei[i]=sum_me
    sum_me=0

KeyError: u'DORpm'

In [129]:
Voxel_structures=pd.DataFrame(structure_tree.get_structures_by_set_id([687527945]))
Voxel_structures[Voxel_structures=='fiber tracts'].index

RangeIndex(start=0, stop=293, step=1)

Recall that we need to remove fiber tracts and ENTmv. So cleam em up

#### BS to TH

In [141]:
Substr[acronym_to_id['TH']]

[856, 864]

In [142]:
Unpack(acronym_to_id['TH'],Substr)

[856,
 51,
 189,
 575,
 599,
 907,
 930,
 560581563,
 138,
 218,
 325,
 1020,
 1029,
 560581551,
 239,
 64,
 127,
 1096,
 1104,
 155,
 255,
 1113,
 1120,
 262,
 444,
 59,
 362,
 366,
 1077,
 571,
 15,
 149,
 181,
 560581559,
 958,
 186,
 483,
 1014,
 27,
 178,
 321,
 563807439,
 864,
 406,
 414,
 422,
 609,
 637,
 685,
 563807435,
 629,
 709,
 718,
 725,
 733,
 741,
 1008,
 170,
 496345664,
 496345668,
 496345672,
 475,
 1072,
 1079,
 1088,
 1044]

In [143]:
Damasio_Network=[['SCs','SCm'],
                ['PB'],
                ['NTS'],
                ['PAG'],
                ['AP'],
                ]
Damasio_Network=Damasio_Network+[id_to_acronym[i] for i in Unpack(acronym_to_id['TH'],Substr)]
#for i in Substr[acronym_to_id['TH']]:
#    toappend=[str(id_to_acronym[j]) for j in Substr[i]]
#    Damasio_Network.append(toappend)

In [146]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[j]=0
            else:
                exception_dict[j]+=1

In [148]:
BS_to_TH={}
for i in flat_Damasio[:6]:
    BS_to_TH[('BS',i)]=0

for i in flat_Damasio[:6]:
    for j in flat_Damasio[6:]:
        if j not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            BS_to_TH[('BS',i)]+=add

In [149]:
BS_to_TH

{('BS', 'AP'): 12460.166371123903,
 ('BS', 'NTS'): 29654.919629143642,
 ('BS', 'PAG'): 34568154.08296166,
 ('BS', 'PB'): 1141972.0820060587,
 ('BS', 'SCm'): 23268523.906333372,
 ('BS', 'SCs'): 6760251.82519597}

In [374]:

BS_to_TH

Pretty printing has been turned ON


{('BS', 'AP'): 12460.166371123903,
 ('BS', 'NTS'): 29654.919629143642,
 ('BS', 'PAG'): 34568154.08296166,
 ('BS', 'PB'): 1141972.0820060587,
 ('BS', 'SCm'): 23268523.906333372,
 ('BS', 'SCs'): 6760251.82519597}

In [170]:
for i in BS_to_TH:
    error=math.sqrt(dreg[belong[i[1]]]**2+dfiring**2+dencode**2)
    value=2*int(BS_to_TH[i])
    print i[1]+" & $\\rightarrow$ &"+'TH'+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

SCs & $\rightarrow$ &TH & 13520502 $\pm$ 7883739 & 58 \\
PB & $\rightarrow$ &TH & 2283944 $\pm$ 1647608 & 72 \\
NTS & $\rightarrow$ &TH & 59308 $\pm$ 35860 & 60 \\
AP & $\rightarrow$ &TH & 24920 $\pm$ 15067 & 60 \\
PAG & $\rightarrow$ &TH & 69136308 $\pm$ 40313048 & 58 \\
SCm & $\rightarrow$ &TH & 46537046 $\pm$ 27135527 & 58 \\


#### TH to BS

In [171]:
exception_dict={}
Damasio_dict={}

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        try:
            proj=Voxel_connectivity_dict[('ipsi',i)][j]*92000
            Damasio_dict[(i,j)]=proj
        except:
            if i not in exception_dict:
                exception_dict[i]=0
            else:
                exception_dict[i]+=1

In [172]:
TH_to_BS={}
for i in flat_Damasio[:6]:
    TH_to_BS[('TH',i)]=0

for i in flat_Damasio[6:]:
    for j in flat_Damasio[:6]:
        if i not in exception_dict.keys():
            add=Damasio_dict[(i,j)]
            TH_to_BS[('TH',j)]+=add

In [173]:
TH_to_BS

{('TH', 'AP'): 679336.9869890029,
 ('TH', 'NTS'): 3525164.047008895,
 ('TH', 'PAG'): 138277715.4782364,
 ('TH', 'PB'): 22941761.250838745,
 ('TH', 'SCm'): 328942051.553909,
 ('TH', 'SCs'): 144937342.76368484}

In [174]:
for i in TH_to_BS:
    error=math.sqrt(0.59**2+dfiring**2+dencode**2)
    value=2*int(TH_to_BS[i])
    print "TH"+" & $\\rightarrow$ &"+str(i[1])+" & "+str(value)+" $\pm$ "+str(int(value*error))+" & "+str(int(100*error))+" \\\\"

TH & $\rightarrow$ &PAG & 276555430 $\pm$ 213879466 & 77 \\
TH & $\rightarrow$ &AP & 1358672 $\pm$ 1050755 & 77 \\
TH & $\rightarrow$ &PB & 45883522 $\pm$ 35484905 & 77 \\
TH & $\rightarrow$ &NTS & 7050328 $\pm$ 5452506 & 77 \\
TH & $\rightarrow$ &SCm & 657884102 $\pm$ 508787335 & 77 \\
TH & $\rightarrow$ &SCs & 289874684 $\pm$ 224180167 & 77 \\


#### Clean em up

In [128]:
indexfinder=Voxel_structures['acronym']
indexfinder[indexfinder=='fiber tracts']

NameError: name 'Voxel_structures' is not defined

In [335]:
indexfinder[indexfinder=='ENTmv']

226    ENTmv
Name: acronym, dtype: object

In [336]:
Voxel_structures=Voxel_structures.drop([247,226]) # drop em bois

In [393]:
Voxel_structures.shape

(291, 8)

nice

In [117]:
"""Substr_Voxel={} # key: structure id, value =[] 
for i in Voxel_structures['acronym']:
    if i not in Substr_Voxel.keys():
        Substr_Voxel[acronym_to_id[i]]=[]


idpath=Voxel_structures['structure_id_path']
for i in idpath:
    print i[-2]
    try:
        Substr_Voxel[i[-2]].append(i[-1])
    except:
        print i"""

"Substr_Voxel={} # key: structure id, value =[] \nfor i in Voxel_structures['acronym']:\n    if i not in Substr_Voxel.keys():\n        Substr_Voxel[acronym_to_id[i]]=[]\n\n\nidpath=Voxel_structures['structure_id_path']\nfor i in idpath:\n    print i[-2]\n    try:\n        Substr_Voxel[i[-2]].append(i[-1])\n    except:\n        print i"

In [409]:
id_to_name[557]

u'Tuberomammillary nucleus'

## Completing Damasio Network

In [398]:
for i in CTX_to_BS:
    print i,round(sum([CTX_to_BS[i][j] for j in CTX_to_BS[i]]),2)

VISC 448146.48
SS 1927754.04
ACA 3959033.84
AUD 414137.81
ILA 869132.09
AI 77975.13
TEa 0.0
MO 2649989.21
GU 1539.87
ECT 137089.71
VIS 10282252.17
FRP 10576.86
PTLp 147662.07
ORB 978561.55
RSP 471877.36
PL 238919.97


In [463]:
Voxel_connectivity_dict[('ipsi','SCm')]['PB']*92000

8485686.30076945

In [458]:
Voxel_connectivity_dict.keys()[550]

('ipsi', 'VCO')

# Homogeneous model

In [1]:
from mcmodels.core import VoxelModelCache, RegionalData
from mcmodels.models import HomogeneousModel

In [6]:
# get data with which to fit model

cache = VoxelModelCache()
voxel_data = cache.get_experiment_data()

MemoryError: 

In [3]:
regional_data = RegionalData.from_voxel_data(voxel_data)

In [ ]:
[
('CTX','HY'),
('CTX','PB'),
('CTX','PAG'),
('CTX','SCm'),
('CTX','SCm')
('HY','PAG'),
('HY','NTS'),
('PAG','SCm'),
('PAG','SCs'),
('PAG','PB'),
('PAG','NTS'),
('PB','AP'),
('PB','NTS')

]

In [4]:
reg = HomogeneousModel()
reg.fit(regional_data.injections, regional_data.projections)
HomogeneousModel(kappa=1000)

HomogeneousModel(kappa=1000)

In [5]:
HomogeneousModel.weights

# mcmodel: mesoscale mouse model data